In [ ]:
# S38: Robust optimizer with larger objective (last-2 blocks and gamma-weighted full) + shrink-to-equal hedge
import numpy as np, pandas as pd, time, os
from sklearn.metrics import roc_auc_score

id_col = 'request_id'; target_col = 'requester_received_pizza'
train = pd.read_json('train.json')
test = pd.read_json('test.json')
y = train[target_col].astype(int).values
ids = test[id_col].values

def to_logit(p, eps=1e-6):
    p = np.clip(p.astype(np.float64), eps, 1.0 - eps)
    return np.log(p / (1.0 - p))
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

# 6-block forward-chaining blocks and masks
order = np.argsort(train['unix_timestamp_of_request'].values)
k = 6
blocks = np.array_split(order, k)
n = len(train)
mask_full = np.zeros(n, dtype=bool)
for bi in range(1, k):
    mask_full[np.array(blocks[bi])] = True
# last two validated blocks (blocks[4], blocks[5])
mask_last2 = np.zeros(n, dtype=bool)
for bi in [4, 5]:
    mask_last2[np.array(blocks[bi])] = True
print(f'Validated counts -> full: {mask_full.sum()}/{n}, last2: {mask_last2.sum()}')

# Load time-consistent base OOF/test predictions (7 bases)
o_lr_w = np.load('oof_lr_time_withsub_meta.npy');   t_lr_w = np.load('test_lr_time_withsub_meta.npy')
o_lr_ns = np.load('oof_lr_time_nosub_meta.npy');    t_lr_ns = np.load('test_lr_time_nosub_meta.npy')
o_d1 = np.load('oof_xgb_dense_time.npy');           t_d1 = np.load('test_xgb_dense_time.npy')
o_d2 = np.load('oof_xgb_dense_time_v2.npy');        t_d2 = np.load('test_xgb_dense_time_v2.npy')
o_meta = np.load('oof_xgb_meta_time.npy');          t_meta = np.load('test_xgb_meta_time.npy')
o_emn = np.load('oof_xgb_emb_meta_time.npy');       t_emn = np.load('test_xgb_emb_meta_time.npy')  # MiniLM
o_emp = np.load('oof_xgb_emb_mpnet_time.npy');      t_emp = np.load('test_xgb_emb_mpnet_time.npy') # MPNet

# Convert to logits
z_lr_w, z_lr_ns = to_logit(o_lr_w), to_logit(o_lr_ns)
z_d1, z_d2, z_meta = to_logit(o_d1), to_logit(o_d2), to_logit(o_meta)
z_emn, z_emp = to_logit(o_emn), to_logit(o_emp)
tz_lr_w, tz_lr_ns = to_logit(t_lr_w), to_logit(t_lr_ns)
tz_d1, tz_d2, tz_meta = to_logit(t_d1), to_logit(t_d2), to_logit(t_meta)
tz_emn, tz_emp = to_logit(t_emn), to_logit(t_emp)

# Search grids (robust, not too wide)
g_grid = [0.94, 0.96, 0.98]  # LR withsub/nosub mix (weight on nosub)
meta_grid = [0.18, 0.20, 0.22]
dense_tot_grid = [0.24, 0.30, 0.35, 0.40]
alpha_grid = [0.50, 0.65, 0.80]  # split dense_total into v2 (w_d2 = d_tot * alpha)
emn_grid = [0.10, 0.12, 0.15]    # MiniLM
emp_grid = [0.08, 0.10, 0.12]    # MPNet

def search_best(mask, sample_weight=None):
    best_auc, best_cfg, tried = -1.0, None, 0
    for g in g_grid:
        z_lr_mix = (1.0 - g)*z_lr_w + g*z_lr_ns
        tz_lr_mix = (1.0 - g)*tz_lr_w + g*tz_lr_ns
        for w_emn in emn_grid:
            for w_emp in emp_grid:
                for w_meta in meta_grid:
                    for d_tot in dense_tot_grid:
                        w_lr = 1.0 - (w_emn + w_emp + w_meta + d_tot)
                        if w_lr <= 0 or w_lr >= 1: continue
                        for a in alpha_grid:
                            w_d2 = d_tot * a; w_d1 = d_tot - w_d2
                            if w_d1 < 0 or w_d2 < 0: continue
                            z_oof = (w_lr*z_lr_mix + w_d1*z_d1 + w_d2*z_d2 + w_meta*z_meta + w_emn*z_emn + w_emp*z_emp)
                            auc = roc_auc_score(y[mask], z_oof[mask], sample_weight=(sample_weight[mask] if sample_weight is not None else None))
                            tried += 1
                            if auc > best_auc:
                                best_auc = auc
                                best_cfg = dict(g=float(g), w_lr=float(w_lr), w_d1=float(w_d1), w_d2=float(w_d2), w_meta=float(w_meta),
                                                w_emn=float(w_emn), w_emp=float(w_emp), tz_lr_mix=tz_lr_mix)
    return best_auc, best_cfg, tried

# Objective A: Gamma-weighted over all validated blocks
best_gamma, best_auc_g, best_cfg_g = None, -1.0, None
for gamma in [0.94, 0.96, 0.98]:
    w = np.zeros(n, dtype=np.float64)
    for bi in range(1, k):
        age = (k - 1) - bi  # newer blocks get higher weight
        w[np.array(blocks[bi])] = (gamma ** age)
    auc_g, cfg_g, tried_g = search_best(mask_full, sample_weight=w)
    print(f'[Gamma {gamma}] tried={tried_g} | best AUC={auc_g:.5f}')
    if auc_g > best_auc_g:
        best_auc_g, best_cfg_g, best_gamma = auc_g, cfg_g, gamma
print(f'[Gamma-best] gamma={best_gamma} | AUC={best_auc_g:.5f} | cfg={{k:v for k,v in best_cfg_g.items() if k!="tz_lr_mix"}}')

# Objective B: Larger validation = blocks 4-5 only
auc_last2, cfg_last2, tried_l2 = search_best(mask_last2)
print(f'[Last2] tried={tried_l2} | best AUC(last2)={auc_last2:.5f} | cfg={{k:v for k,v in cfg_last2.items() if k!="tz_lr_mix"}}')

# Choose primary objective: prefer gamma-best for robustness; keep last2 as alternate
primary_tag = f'gamma{best_gamma:.2f}'.replace('.', 'p')
primary_cfg = best_cfg_g
alt_tag = 'last45'
alt_cfg = cfg_last2

def build_submission(tag, cfg, do_shrink=True, alphas=(0.10, 0.15, 0.20)):
    tz_lr_mix = cfg['tz_lr_mix']
    w_lr, w_d1, w_d2, w_meta, w_emn, w_emp = cfg['w_lr'], cfg['w_d1'], cfg['w_d2'], cfg['w_meta'], cfg['w_emn'], cfg['w_emp']
    zt = (w_lr*tz_lr_mix + w_d1*tz_d1 + w_d2*tz_d2 + w_meta*tz_meta + w_emn*tz_emn + w_emp*tz_emp)
    pt = sigmoid(zt).astype(np.float32)
    pd.DataFrame({id_col: ids, target_col: pt}).to_csv(f'submission_s38_{tag}.csv', index=False)
    print(f'Wrote submission_s38_{tag}.csv | mean={pt.mean():.6f}')
    best_shr_auc, best_alpha, best_pt = -1.0, None, None
    if do_shrink:
        w_vec = np.array([w_lr, w_d1, w_d2, w_meta, w_emn, w_emp], dtype=np.float64)
        w_eq = np.ones_like(w_vec) / len(w_vec)
        # Evaluate shrink on gamma-weighted full mask for stability
        w_sw = np.zeros(n, dtype=np.float64)
        for bi in range(1, k):
            age = (k - 1) - bi
            w_sw[np.array(blocks[bi])] = (best_gamma ** age) if best_gamma is not None else 1.0
        for a in alphas:
            w_shr = ((1.0 - a)*w_vec + a*w_eq); w_shr = (w_shr / w_shr.sum()).astype(np.float64)
            z_oof = (w_shr[0]*((1.0 - primary_cfg['g'])*z_lr_w + primary_cfg['g']*z_lr_ns) +
                     w_shr[1]*z_d1 + w_shr[2]*z_d2 + w_shr[3]*z_meta + w_shr[4]*z_emn + w_shr[5]*z_emp)
            auc = roc_auc_score(y[mask_full], z_oof[mask_full], sample_weight=w_sw[mask_full])
            if auc > best_shr_auc:
                best_shr_auc, best_alpha = auc, a
                zt_shr = (w_shr[0]*tz_lr_mix + w_shr[1]*tz_d1 + w_shr[2]*tz_d2 + w_shr[3]*tz_meta + w_shr[4]*tz_emn + w_shr[5]*tz_emp)
                best_pt = sigmoid(zt_shr).astype(np.float32)
        if best_alpha is not None:
            fn = f'submission_s38_{tag}_shrunk_a{int(best_alpha*100)}.csv'
            pd.DataFrame({id_col: ids, target_col: best_pt}).to_csv(fn, index=False)
            print(f'Wrote {fn} | alpha={best_alpha:.2f} | mean={best_pt.mean():.6f} | shr_AUC={best_shr_auc:.5f}')
            return pt, best_pt, best_alpha
    return pt, None, None

# Build primary and alternate submissions
pt_primary, pt_primary_shr, alpha_primary = build_submission(primary_tag, primary_cfg, do_shrink=True)
pt_alt, pt_alt_shr, alpha_alt = build_submission(alt_tag, alt_cfg, do_shrink=True)

# Promote conservative hedge: gamma-best shrunk if available, else gamma-best raw
promote_path = f'submission_s38_{primary_tag}.csv'
if pt_primary_shr is not None:
    promote_path = f'submission_s38_{primary_tag}_shrunk_a{int(alpha_primary*100)}.csv'
pd.read_csv(promote_path).to_csv('submission.csv', index=False)
print('Promoted to submission.csv ->', promote_path)

In [ ]:
# S39: Gamma-weighted optimizer with small recent alphas (LR_nosub + MiniLM) + optional CatBoost + shrink + hedges
import numpy as np, pandas as pd, os, time
from sklearn.metrics import roc_auc_score

id_col = 'request_id'; target_col = 'requester_received_pizza'
train = pd.read_json('train.json'); test = pd.read_json('test.json')
y = train[target_col].astype(int).values; ids = test[id_col].values

def to_logit(p, eps=1e-6):
    p = np.clip(p.astype(np.float64), eps, 1.0 - eps)
    return np.log(p / (1.0 - p))
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

# Time blocks
order = np.argsort(train['unix_timestamp_of_request'].values)
k = 6
blocks = np.array_split(order, k)
n = len(train)
mask_full = np.zeros(n, bool)
for bi in range(1, k):
    mask_full[np.array(blocks[bi])] = True

# Load base OOF/test predictions (same 7 as S38); optional CatBoost
o_lr_w = np.load('oof_lr_time_withsub_meta.npy');   t_lr_w = np.load('test_lr_time_withsub_meta.npy')
o_lr_ns = np.load('oof_lr_time_nosub_meta.npy');    t_lr_ns = np.load('test_lr_time_nosub_meta.npy')
o_d1 = np.load('oof_xgb_dense_time.npy');           t_d1 = np.load('test_xgb_dense_time.npy')
o_d2 = np.load('oof_xgb_dense_time_v2.npy');        t_d2 = np.load('test_xgb_dense_time_v2.npy')
o_meta = np.load('oof_xgb_meta_time.npy');          t_meta = np.load('test_xgb_meta_time.npy')
o_emn = np.load('oof_xgb_emb_meta_time.npy');       t_emn = np.load('test_xgb_emb_meta_time.npy')  # MiniLM
o_emp = np.load('oof_xgb_emb_mpnet_time.npy');      t_emp = np.load('test_xgb_emb_mpnet_time.npy') # MPNet

has_cat = os.path.exists('oof_catboost_textmeta_v2.npy') and os.path.exists('test_catboost_textmeta_v2.npy')
if has_cat:
    o_cat = np.load('oof_catboost_textmeta_v2.npy'); t_cat = np.load('test_catboost_textmeta_v2.npy')

# Logits
z_lr_w, z_lr_ns = to_logit(o_lr_w), to_logit(o_lr_ns)
z_d1, z_d2, z_meta = to_logit(o_d1), to_logit(o_d2), to_logit(o_meta)
z_emn, z_emp = to_logit(o_emn), to_logit(o_emp)
tz_lr_w, tz_lr_ns = to_logit(t_lr_w), to_logit(t_lr_ns)
tz_d1, tz_d2, tz_meta = to_logit(t_d1), to_logit(t_d2), to_logit(t_meta)
tz_emn, tz_emp = to_logit(t_emn), to_logit(t_emp)
if has_cat:
    z_cat, tz_cat = to_logit(o_cat), to_logit(t_cat)

# Load recent test-only logits (average of recent35/45) for LR_nosub and MiniLM only
def load_recent_avg_logit(prefix):
    arrs = []
    for suf in ['_recent35.npy', '_recent45.npy']:
        p = prefix + suf
        if os.path.exists(p):
            try:
                arrs.append(to_logit(np.load(p)))
            except Exception:
                pass
    return np.mean(arrs, axis=0).astype(np.float64) if arrs else None

# Recent sources discovered previously
tz_lr_ns_r = load_recent_avg_logit('test_lr_nosub_meta')  # LR no-sub recent
tz_emn_r = load_recent_avg_logit('test_xgb_minilm_meta')  # MiniLM recent
print('Recent availability:', {'lr_nosub': tz_lr_ns_r is not None, 'minilm': tz_emn_r is not None})

# Grids per expert advice
g_list = [0.96, 0.98, 0.99]  # LR mix
dense_tot_grid = [0.18, 0.22, 0.26, 0.30]
v2_frac_grid = [0.55, 0.65, 0.75]
meta_grid = [0.18, 0.20, 0.22]
emn_grid = [0.10, 0.12, 0.14]
emp_grid = [0.08, 0.10, 0.12]
cat_grid = [0.04, 0.06] if has_cat else [0.0]
gamma_list = [0.96, 0.98, 0.995]
shrink_list = [0.15, 0.20, 0.25]

# Build gamma sample weights with optional double weight on newest validated block (block 5)
def gamma_weights(gamma, double_last=True):
    w = np.zeros(n, np.float64)
    for bi in range(1, k):
        age = (k - 1) - bi
        w_block = (gamma ** age)
        if double_last and bi == 5:
            w_block *= 2.0
        w[np.array(blocks[bi])] = w_block
    return w

def search_gamma_primary():
    best_auc, best_cfg, best_gamma = -1.0, None, None
    total_tried = 0
    for gamma in gamma_list:
        w = gamma_weights(gamma, double_last=True)
        best_auc_g = -1.0; best_cfg_g = None; tried = 0
        for g in g_list:
            z_lr_mix = (1.0 - g) * z_lr_w + g * z_lr_ns
            for d_tot in dense_tot_grid:
                for v2f in v2_frac_grid:
                    w_d2 = d_tot * v2f; w_d1 = d_tot - w_d2
                    if w_d1 < 0 or w_d2 < 0: continue
                    for w_meta in meta_grid:
                        for w_emn in emn_grid:
                            for w_emp in emp_grid:
                                for w_cat in cat_grid:
                                    w_sum = d_tot + w_meta + w_emn + w_emp + (w_cat if has_cat else 0.0)
                                    w_lr = 1.0 - w_sum
                                    if (w_lr <= 0) or (w_lr >= 1):
                                        continue
                                    # Enforce LRmix floor/ceiling
                                    if not (0.28 <= w_lr <= 0.50):
                                        continue
                                    # Cap CatBoost (already grid-capped), ensure non-negative remaining weights
                                    z = (w_lr * z_lr_mix + w_d1 * z_d1 + w_d2 * z_d2 + w_meta * z_meta +
                                         w_emn * z_emn + w_emp * z_emp + ((w_cat * z_cat) if has_cat else 0.0))
                                    auc = roc_auc_score(y[mask_full], z[mask_full], sample_weight=w[mask_full])
                                    tried += 1; total_tried += 1
                                    if auc > best_auc_g:
                                        best_auc_g = auc
                                        best_cfg_g = dict(g=float(g), w_lr=float(w_lr), w_d1=float(w_d1), w_d2=float(w_d2),
                                                          w_meta=float(w_meta), w_emn=float(w_emn), w_emp=float(w_emp))
                                        if has_cat: best_cfg_g['w_cat'] = float(w_cat)
        print(f'[Gamma {gamma}] tried={tried} | best AUC={best_auc_g:.5f}')
        if best_auc_g > best_auc:
            best_auc, best_cfg, best_gamma = best_auc_g, best_cfg_g, gamma
    print(f'[Gamma-primary] gamma={best_gamma} | AUC={best_auc:.5f} | cfg={{k:v for k,v in best_cfg.items()}}')
    return best_gamma, best_auc, best_cfg

best_gamma, best_auc, best_cfg = search_gamma_primary()

# Build test logits for a given cfg, with optional recent interpolation
def build_test_logits(cfg, a_lr_ns=0.0, a_mn=0.0):
    g = cfg['g']
    # Full-history logits
    tz_lr_mix_full = (1.0 - g) * tz_lr_w + g * tz_lr_ns
    tz_mn_full = tz_emn
    # Recent targets
    tz_lr_ns_recent = tz_lr_ns_r if tz_lr_ns_r is not None else tz_lr_ns
    tz_lr_mix_recent = (1.0 - g) * tz_lr_w + g * tz_lr_ns_recent
    tz_mn_recent = tz_emn_r if tz_emn_r is not None else tz_emn
    # Apply small alphas
    tz_lr_mix = (1.0 - a_lr_ns) * tz_lr_mix_full + a_lr_ns * tz_lr_mix_recent
    tz_mn = (1.0 - a_mn) * tz_mn_full + a_mn * tz_mn_recent
    # Combine
    parts = [ (cfg['w_lr'], tz_lr_mix), (cfg['w_d1'], tz_d1), (cfg['w_d2'], tz_d2), (cfg['w_meta'], tz_meta),
              (cfg['w_emn'], tz_mn), (cfg['w_emp'], tz_emp) ]
    if has_cat:
        parts.append((cfg.get('w_cat', 0.0), tz_cat))
    zt = np.zeros_like(tz_d1, dtype=np.float64)
    for wv, zz in parts:
        zt += wv * zz
    return zt

# Evaluate shrink on OOF (no recents) under gamma weights; choose best alpha
def apply_shrink_and_choose(cfg):
    w_vec = np.array([cfg['w_lr'], cfg['w_d1'], cfg['w_d2'], cfg['w_meta'], cfg['w_emn'], cfg['w_emp']] + ([cfg['w_cat']] if has_cat else []), dtype=np.float64)
    w_eq = np.ones_like(w_vec) / len(w_vec)
    g = cfg['g']
    z_lr_mix = (1.0 - g) * z_lr_w + g * z_lr_ns
    parts = [z_lr_mix, z_d1, z_d2, z_meta, z_emn, z_emp] + ([z_cat] if has_cat else [])
    W = gamma_weights(best_gamma, double_last=True)
    best_a, best_auc = None, -1.0
    for a in shrink_list:
        w_shr = ((1.0 - a) * w_vec + a * w_eq)
        w_shr = (w_shr / w_shr.sum()).astype(np.float64)
        z = np.zeros_like(z_d1, dtype=np.float64)
        for wi, zi in zip(w_shr, parts):
            z += wi * zi
        auc = roc_auc_score(y[mask_full], z[mask_full], sample_weight=W[mask_full])
        if auc > best_auc:
            best_auc, best_a, best_w = auc, a, w_shr.copy()
    return best_a, best_w, best_auc

best_shr_alpha, best_shr_w, shr_auc = apply_shrink_and_choose(best_cfg)
print(f'[Shrink] best alpha={best_shr_alpha:.2f} | shr_AUC={shr_auc:.5f}')

# Build and save gamma-shrunk submission (no recent interpolation)
def build_from_weights_vec(w_vec, cfg):
    g = cfg['g']
    tz_lr_mix = (1.0 - g) * tz_lr_w + g * tz_lr_ns
    parts = [tz_lr_mix, tz_d1, tz_d2, tz_meta, tz_emn, tz_emp] + ([tz_cat] if has_cat else [])
    zt = np.zeros_like(tz_d1, dtype=np.float64)
    for wi, zi in zip(w_vec, parts):
        zt += wi * zi
    return zt

# Map shrunk vector back to named cfg for convenience
def named_cfg_from_shrunk(w_vec, cfg):
    keys = ['w_lr','w_d1','w_d2','w_meta','w_emn','w_emp'] + (['w_cat'] if has_cat else [])
    out = dict(cfg)
    for key, val in zip(keys, w_vec):
        out[key] = float(val)
    return out

# Save gamma-shrunk
keys = ['w_lr','w_d1','w_d2','w_meta','w_emn','w_emp'] + (['w_cat'] if has_cat else [])
base_w = np.array([best_cfg[k] if k in best_cfg else 0.0 for k in keys], dtype=np.float64)
w_gamma_shr = best_shr_w
zt_gamma_shr = build_from_weights_vec(w_gamma_shr, best_cfg)
p_gamma_shr = sigmoid(zt_gamma_shr).astype(np.float32)
fn_gamma = f'submission_s39_gamma_shrunk_a{int(best_shr_alpha*100)}.csv'
pd.DataFrame({id_col: ids, target_col: p_gamma_shr}).to_csv(fn_gamma, index=False)
print('Wrote', fn_gamma, '| mean', float(p_gamma_shr.mean()))

# Recent alpha search: small alphas in {0, .05, .10, .15} for LR_nosub and MiniLM; enforce r≈0.24 and r≈0.30
a_grid = [0.0, 0.05, 0.10, 0.15]
def pick_recent_alphas(cfg, r_target, tol=0.02):
    # Effective recent share r ≈ w_lrmix * g * a_lr_ns + w_emn * a_mn
    g = cfg['g']
    w_lr = cfg['w_lr']; w_mn = cfg['w_emn']
    best = (0.0, 0.0); best_err = 1e9; best_sum = 1e9
    for a_lr_ns in a_grid:
        for a_mn in a_grid:
            # If recent artifact missing, force alpha=0 for that model
            if tz_lr_ns_r is None and a_lr_ns > 0: continue
            if tz_emn_r is None and a_mn > 0: continue
            r_est = w_lr * g * a_lr_ns + w_mn * a_mn
            err = abs(r_est - r_target); sm = a_lr_ns + a_mn
            if (err < best_err) or (abs(err - best_err) < 1e-12 and sm < best_sum):
                best_err, best_sum, best = err, sm, (a_lr_ns, a_mn)
    return best

# Build r~0.24 and r~0.30 with the gamma-shrunk weights
cfg_gamma_shr = named_cfg_from_shrunk(w_gamma_shr, best_cfg)
a24 = pick_recent_alphas(cfg_gamma_shr, 0.24)
a30 = pick_recent_alphas(cfg_gamma_shr, 0.30)
print('Chosen recent alphas: r24', a24, '| r30', a30)

zt_r24 = build_test_logits(cfg_gamma_shr, a_lr_ns=a24[0], a_mn=a24[1])
p_r24 = sigmoid(zt_r24).astype(np.float32)
fn_r24 = f'submission_s39_r24_shrunk_a{int(best_shr_alpha*100)}.csv'
pd.DataFrame({id_col: ids, target_col: p_r24}).to_csv(fn_r24, index=False)
print('Wrote', fn_r24, '| mean', float(p_r24.mean()))

zt_r30 = build_test_logits(cfg_gamma_shr, a_lr_ns=a30[0], a_mn=a30[1])
p_r30 = sigmoid(zt_r30).astype(np.float32)
fn_r30 = f'submission_s39_r30_shrunk_a{int(best_shr_alpha*100)}.csv'
pd.DataFrame({id_col: ids, target_col: p_r30}).to_csv(fn_r30, index=False)
print('Wrote', fn_r30, '| mean', float(p_r30.mean()))

# Hedged submissions: 2-way (gamma_shrunk, r24_shrunk) and 3-way (+ r30_shrunk)
def logit_avg(paths, out):
    zs = [to_logit(pd.read_csv(p)[target_col].values.astype(np.float64)) for p in paths]
    z = np.mean(zs, axis=0); p = sigmoid(z).astype(np.float32)
    pd.DataFrame({id_col: ids, target_col: p}).to_csv(out, index=False)
    print('Wrote', out, '| mean', float(p.mean()))

fn_hedge2 = 'submission_s39_hedge2_gamma_r24.csv'
logit_avg([fn_gamma, fn_r24], fn_hedge2)
fn_hedge3 = 'submission_s39_hedge3_gamma_r24_r30.csv'
logit_avg([fn_gamma, fn_r24, fn_r30], fn_hedge3)

# Promote safest hedge (2-way) to submission.csv
pd.read_csv(fn_hedge2).to_csv('submission.csv', index=False)
print('Promoted', fn_hedge2, 'to submission.csv')

In [ ]:
# S39-cat: Generate full-test predictions for CatBoost text+meta v2 (to enable has_cat in S39)
import numpy as np, pandas as pd, os, sys, time
from catboost import CatBoostClassifier, Pool

id_col = 'request_id'; target_col = 'requester_received_pizza'
train = pd.read_json('train.json'); test = pd.read_json('test.json')

def get_text(df: pd.DataFrame) -> pd.Series:
    title = df.get('request_title', pd.Series(['']*len(df))).fillna('').astype(str)
    body = df.get('request_text_edit_aware', df.get('request_text', pd.Series(['']*len(df)))).fillna('').astype(str)
    return (title + '\n' + body)

# Build feature frame: one text column + numeric metas present in BOTH train and test
txt_tr = get_text(train); txt_te = get_text(test)
drop_cols = {target_col, id_col, 'request_title', 'request_text', 'request_text_edit_aware', 'unix_timestamp_of_request'}
train_num = [c for c in train.columns if (c not in drop_cols) and pd.api.types.is_numeric_dtype(train[c])]
test_num = [c for c in test.columns if (c not in drop_cols) and pd.api.types.is_numeric_dtype(test[c])]
num_cols = sorted(set(train_num).intersection(test_num))
print('Numeric cols used (intersection):', len(num_cols))
X_tr = pd.concat([txt_tr.rename('txt'), train[num_cols]], axis=1)
X_te = pd.concat([txt_te.rename('txt'), test[num_cols]], axis=1)
y = train[target_col].astype(int).values

text_features = [0]  # first column is 'txt'
train_pool = Pool(X_tr, label=y, text_features=text_features)
test_pool = Pool(X_te, text_features=text_features)

params = dict(
    loss_function='Logloss',
    eval_metric='AUC',
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=5.0,
    random_seed=42,
    iterations=700,
    task_type='GPU',
    devices='0',
    verbose=100
)

print('Training CatBoost text+meta v2 on full train...')
t0 = time.time()
model = CatBoostClassifier(**params)
model.fit(train_pool)
print(f'Training done in {time.time()-t0:.1f}s')

print('Predicting test probabilities...')
proba = model.predict_proba(test_pool)[:,1].astype(np.float32)
np.save('test_catboost_textmeta_v2.npy', proba)
print('Saved test_catboost_textmeta_v2.npy | mean=', float(proba.mean()))

In [ ]:
# S40: Time-aware L2 stacker on base logits + small recency alphas + hedges
import numpy as np, pandas as pd, os, time
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

id_col='request_id'; target_col='requester_received_pizza'
train=pd.read_json('train.json'); test=pd.read_json('test.json')
y=train[target_col].astype(int).values; ids=test[id_col].values

def to_logit(p, eps=1e-6):
    p = np.clip(p.astype(np.float64), eps, 1.0-eps)
    return np.log(p/(1.0-p))
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

# 6-block forward-chaining masks and gamma weights
order=np.argsort(train['unix_timestamp_of_request'].values); k=6; blocks=np.array_split(order,k); n=len(train)
mask_full=np.zeros(n,bool)
for bi in range(1,k): mask_full[np.array(blocks[bi])]=True
def gamma_weights(gamma=0.995, double_last=True):
    w=np.zeros(n,np.float64)
    for bi in range(1,k):
        age=(k-1)-bi; wb=(gamma**age)
        if double_last and bi==5: wb*=2.0
        w[np.array(blocks[bi])]=wb
    return w
W=gamma_weights(0.995, double_last=True)

# Load base OOF/test probs, convert to logits; columns in fixed order
paths_oof=[
 'oof_lr_time_withsub_meta.npy',
 'oof_lr_time_nosub_meta.npy',
 'oof_xgb_dense_time.npy',
 'oof_xgb_dense_time_v2.npy',
 'oof_xgb_meta_time.npy',
 'oof_xgb_emb_meta_time.npy',
 'oof_xgb_emb_mpnet_time.npy'
]
paths_te=[
 'test_lr_time_withsub_meta.npy',
 'test_lr_time_nosub_meta.npy',
 'test_xgb_dense_time.npy',
 'test_xgb_dense_time_v2.npy',
 'test_xgb_meta_time.npy',
 'test_xgb_emb_meta_time.npy',
 'test_xgb_emb_mpnet_time.npy'
]
names=['LR_withsub','LR_nosub','Dense_v1','Dense_v2','Meta','MiniLM','MPNet']

# Optional CatBoost column if OOF exists
has_cat=os.path.exists('oof_catboost_textmeta_v2.npy') and os.path.exists('test_catboost_textmeta_v2.npy')
if has_cat:
    paths_oof.append('oof_catboost_textmeta_v2.npy'); paths_te.append('test_catboost_textmeta_v2.npy'); names.append('CatBoost')

oof_list=[to_logit(np.load(p)) for p in paths_oof]
te_list=[to_logit(np.load(p)) for p in paths_te]
X_oof=np.column_stack(oof_list).astype(np.float64)
X_te_full=np.column_stack(te_list).astype(np.float64)
print('Stack columns:', names)

# Standardize columns
sc=StandardScaler(with_mean=True, with_std=True)
Xz=sc.fit_transform(X_oof)
Xz_te=sc.transform(X_te_full)

# Tune C via gamma-weighted OOF AUC on mask_full
C_grid=[0.05, 0.1, 0.3, 0.5, 1.0, 1.5, 3.0]
best_auc=-1.0; best_C=None; best_model=None
t0=time.time()
for C in C_grid:
    lr=LogisticRegression(penalty='l2', solver='lbfgs', C=C, max_iter=2000, fit_intercept=True)
    lr.fit(Xz[mask_full], y[mask_full], sample_weight=W[mask_full])
    z=lr.decision_function(Xz[mask_full])
    auc=roc_auc_score(y[mask_full], z, sample_weight=W[mask_full])
    print(f'C={C} | gamma-weighted OOF AUC={auc:.5f}')
    if auc>best_auc: best_auc, best_C, best_model=auc, C, lr
print(f'Best C={best_C} | AUC={best_auc:.5f} | time={time.time()-t0:.1f}s')
coef=best_model.coef_.ravel(); intercept=float(best_model.intercept_[0])
print('Coef (name, coef):', list(zip(names, coef)))

# Build meta_gamma test logits/probs (no recency)
z_te_gamma = (Xz_te @ coef) + intercept
p_meta_gamma = sigmoid(z_te_gamma).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p_meta_gamma}).to_csv('submission_s40_meta_gamma.csv', index=False)
print('Wrote submission_s40_meta_gamma.csv | mean', float(p_meta_gamma.mean()))

# Load recent test-only logits for LR_nosub and MiniLM (average of 35/45) if available
def load_recent_avg_logit(prefix):
    arrs=[]
    for suf in ['_recent35.npy','_recent45.npy']:
        p=prefix+suf
        if os.path.exists(p):
            try: arrs.append(to_logit(np.load(p)))
            except: pass
    return np.mean(arrs,axis=0).astype(np.float64) if arrs else None

# Identify column indices
idx_lr_ns = names.index('LR_nosub') if 'LR_nosub' in names else None
idx_minilm = names.index('MiniLM') if 'MiniLM' in names else None
tz_lr_ns_r = load_recent_avg_logit('test_lr_nosub_meta') if idx_lr_ns is not None else None
tz_emn_r   = load_recent_avg_logit('test_xgb_minilm_meta') if idx_minilm is not None else None
print('Recent present:', {'LR_nosub': tz_lr_ns_r is not None, 'MiniLM': tz_emn_r is not None})

# Helper to rebuild transformed X_test with small alphas on raw-logit columns, then re-standardize
a_grid=[0.0, 0.05, 0.10, 0.15]
def build_Xz_te_with_recency(a_lr_ns, a_mn):
    X_mod = X_te_full.copy()
    if (idx_lr_ns is not None) and (tz_lr_ns_r is not None):
        X_mod[:, idx_lr_ns] = (1.0 - a_lr_ns)*X_te_full[:, idx_lr_ns] + a_lr_ns*tz_lr_ns_r
    if (idx_minilm is not None) and (tz_emn_r is not None):
        X_mod[:, idx_minilm] = (1.0 - a_mn)*X_te_full[:, idx_minilm] + a_mn*tz_emn_r
    return sc.transform(X_mod)

# Pick alphas to target r ≈ 0.24 and 0.30 using normalized |coef| shares on those two columns
def pick_alphas(target_r):
    s_lr = abs(coef[idx_lr_ns]) if (idx_lr_ns is not None) else 0.0
    s_mn = abs(coef[idx_minilm]) if (idx_minilm is not None) else 0.0
    if (s_lr+s_mn) <= 0: s_lr=s_mn=1.0
    s_lr /= (s_lr+s_mn); s_mn = 1.0 - s_lr
    best=(0.0,0.0); best_err=1e9; best_sum=1e9
    for a_lr in a_grid:
        for a_mn in a_grid:
            if (tz_lr_ns_r is None) and a_lr>0: continue
            if (tz_emn_r is None) and a_mn>0: continue
            r_est = s_lr*a_lr + s_mn*a_mn
            err = abs(r_est - target_r); sm=a_lr+a_mn
            if (err<best_err) or (abs(err-best_err)<1e-12 and sm<best_sum):
                best_err, best_sum, best = err, sm, (a_lr, a_mn)
    return best

a24 = pick_alphas(0.24)
a30 = pick_alphas(0.30)
print('Chosen alphas -> r24:', a24, '| r30:', a30)

# Build meta_r24 and meta_r30
Xz_te_r24 = build_Xz_te_with_recency(a24[0], a24[1])
z_te_r24 = (Xz_te_r24 @ coef) + intercept
p_meta_r24 = sigmoid(z_te_r24).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p_meta_r24}).to_csv('submission_s40_meta_r24.csv', index=False)
print('Wrote submission_s40_meta_r24.csv | mean', float(p_meta_r24.mean()))

Xz_te_r30 = build_Xz_te_with_recency(a30[0], a30[1])
z_te_r30 = (Xz_te_r30 @ coef) + intercept
p_meta_r30 = sigmoid(z_te_r30).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p_meta_r30}).to_csv('submission_s40_meta_r30.csv', index=False)
print('Wrote submission_s40_meta_r30.csv | mean', float(p_meta_r30.mean()))

# Hedged submissions: 2-way and 3-way logit averages
def logit_avg(paths, out):
    zs=[to_logit(pd.read_csv(p)[target_col].values.astype(np.float64)) for p in paths]
    z=np.mean(zs,axis=0); p=sigmoid(z).astype(np.float32)
    pd.DataFrame({id_col: ids, target_col: p}).to_csv(out, index=False)
    print('Wrote', out, '| mean', float(p.mean()))

logit_avg(['submission_s40_meta_gamma.csv','submission_s40_meta_r24.csv'], 'submission_s40_meta_hedge2.csv')
logit_avg(['submission_s40_meta_gamma.csv','submission_s40_meta_r24.csv','submission_s40_meta_r30.csv'], 'submission_s40_meta_hedge3.csv')

# Optional robustness hedge: 0.85*meta_gamma + 0.15*eq-weight base logits
z_eq = np.mean(X_te_full, axis=1)  # equal-weight average of base logits
z_meta = z_te_gamma
p_robust = sigmoid(0.85*z_meta + 0.15*z_eq).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p_robust}).to_csv('submission_s40_meta_robust.csv', index=False)
print('Wrote submission_s40_meta_robust.csv | mean', float(p_robust.mean()))

# Promote 2-way hedged stacker to submission.csv
pd.read_csv('submission_s40_meta_hedge2.csv').to_csv('submission.csv', index=False)
print('Promoted submission_s40_meta_hedge2.csv to submission.csv')

In [ ]:
# Promote safer S39 hedge2 to submission.csv
import pandas as pd
src = 'submission_s39_hedge2_gamma_r24.csv'
df = pd.read_csv(src)
df.to_csv('submission.csv', index=False)
print('Promoted', src, 'to submission.csv | mean =', float(df['requester_received_pizza'].mean()))

In [ ]:
# S40b: Stacker variants (reduced search) -> pick best gamma-weighted OOF, rebuild submissions
import numpy as np, pandas as pd, os, time
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

id_col='request_id'; target_col='requester_received_pizza'
train=pd.read_json('train.json'); test=pd.read_json('test.json')
y=train[target_col].astype(int).values; ids=test[id_col].values

def to_logit(p, eps=1e-6):
    p = np.clip(p.astype(np.float64), eps, 1.0-eps)
    return np.log(p/(1.0-p))
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

# Time masks and gamma weights
order=np.argsort(train['unix_timestamp_of_request'].values); k=6; blocks=np.array_split(order,k); n=len(train)
mask_full=np.zeros(n,bool)
for bi in range(1,k): mask_full[np.array(blocks[bi])]=True
def gamma_weights(gamma=0.995, double_last=True):
    w=np.zeros(n,np.float64)
    for bi in range(1,k):
        age=(k-1)-bi; wb=(gamma**age)
        if double_last and bi==5: wb*=2.0
        w[np.array(blocks[bi])]=wb
    return w
W=gamma_weights(0.995, double_last=True)

# Base arrays (order fixed); exclude CatBoost to avoid instability
base_oof_paths=[
 'oof_lr_time_withsub_meta.npy',
 'oof_lr_time_nosub_meta.npy',
 'oof_xgb_dense_time.npy',
 'oof_xgb_dense_time_v2.npy',
 'oof_xgb_meta_time.npy',
 'oof_xgb_emb_meta_time.npy',
 'oof_xgb_emb_mpnet_time.npy'
]
base_te_paths=[
 'test_lr_time_withsub_meta.npy',
 'test_lr_time_nosub_meta.npy',
 'test_xgb_dense_time.npy',
 'test_xgb_dense_time_v2.npy',
 'test_xgb_meta_time.npy',
 'test_xgb_emb_meta_time.npy',
 'test_xgb_emb_mpnet_time.npy'
]
base_names=['LR_withsub','LR_nosub','Dense_v1','Dense_v2','Meta','MiniLM','MPNet']

X_oof_full=np.column_stack([to_logit(np.load(p)) for p in base_oof_paths]).astype(np.float64)
X_te_full=np.column_stack([to_logit(np.load(p)) for p in base_te_paths]).astype(np.float64)
print('Stack variants (reduced) over columns:', base_names)

def fit_eval_stack(X_tr, y_tr, W_tr, use_scaler, C_grid):
    if use_scaler:
        sc=StandardScaler(with_mean=True, with_std=True); Xz=sc.fit_transform(X_tr)
    else:
        sc=None; Xz=X_tr
    best_auc=-1; best_C=None; best_model=None
    for C in C_grid:
        lr=LogisticRegression(penalty='l2', solver='lbfgs', C=C, max_iter=2000, fit_intercept=True)
        lr.fit(Xz, y_tr, sample_weight=W_tr)
        z=lr.decision_function(Xz)
        auc=roc_auc_score(y_tr, z, sample_weight=W_tr)
        if auc>best_auc: best_auc, best_C, best_model=auc, C, lr
    return best_auc, best_C, best_model, sc

C_grid=[0.1,0.3,1.0,3.0]
variants=[]
for use_scaler in [True, False]:
    auc, Cbest, model, sc = fit_eval_stack(X_oof_full[mask_full], y[mask_full], W[mask_full], use_scaler, C_grid)
    variants.append(dict(auc=auc, scaler=use_scaler, C=Cbest, model=model, sc=sc))
    print(f'Variant scaler={use_scaler} | AUC={auc:.5f} | C={Cbest}')

best = max(variants, key=lambda d: d['auc'])
print('Best variant:', {k: (v if k not in ['model','sc'] else '...') for k,v in best.items()})

# Build submissions for best variant
model=best['model']; sc=best['sc']
Xz_te = sc.transform(X_te_full) if sc is not None else X_te_full
coef = model.coef_.ravel(); intercept=float(model.intercept_[0])
print('Chosen coef len:', len(coef), '| intercept:', intercept)

# meta_gamma
z_te_gamma = (Xz_te @ coef) + intercept
p_meta_gamma = sigmoid(z_te_gamma).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p_meta_gamma}).to_csv('submission_s40b_meta_gamma.csv', index=False)
print('Wrote submission_s40b_meta_gamma.csv | mean', float(p_meta_gamma.mean()))

# Recency (LR_nosub + MiniLM) on raw-logit space then transform with scaler if any
def load_recent_avg_logit(prefix):
    arrs=[]
    for suf in ['_recent35.npy','_recent45.npy']:
        p=prefix+suf
        if os.path.exists(p):
            try: arrs.append(to_logit(np.load(p)))
            except: pass
    return np.mean(arrs,axis=0).astype(np.float64) if arrs else None

idx_lr_ns = base_names.index('LR_nosub')
idx_minilm = base_names.index('MiniLM')
tz_lr_ns_r = load_recent_avg_logit('test_lr_nosub_meta')
tz_emn_r   = load_recent_avg_logit('test_xgb_minilm_meta')
print('Recent present:', {'LR_nosub': tz_lr_ns_r is not None, 'MiniLM': tz_emn_r is not None})

a_grid=[0.0,0.05,0.10,0.15]
def build_Xz_te_with_recency(a_lr_ns, a_mn):
    X_mod = X_te_full.copy()
    if tz_lr_ns_r is not None:
        X_mod[:, idx_lr_ns] = (1.0 - a_lr_ns)*X_te_full[:, idx_lr_ns] + a_lr_ns*tz_lr_ns_r
    if tz_emn_r is not None:
        X_mod[:, idx_minilm] = (1.0 - a_mn)*X_te_full[:, idx_minilm] + a_mn*tz_emn_r
    return (sc.transform(X_mod) if sc is not None else X_mod)

def pick_alphas(target_r):
    s_lr = abs(coef[idx_lr_ns]); s_mn = abs(coef[idx_minilm])
    if (s_lr+s_mn) <= 0: s_lr=s_mn=1.0
    s_lr /= (s_lr+s_mn); s_mn = 1.0 - s_lr
    best=(0.0,0.0); best_err=1e9; best_sum=1e9
    for a_lr in a_grid:
        for a_mn in a_grid:
            if (tz_lr_ns_r is None) and a_lr>0: continue
            if (tz_emn_r is None) and a_mn>0: continue
            r_est = s_lr*a_lr + s_mn*a_mn
            err = abs(r_est - target_r); sm=a_lr+a_mn
            if (err<best_err) or (abs(err-best_err)<1e-12 and sm<best_sum):
                best_err, best_sum, best = err, sm, (a_lr, a_mn)
    return best

a24 = pick_alphas(0.24); a30 = pick_alphas(0.30)
print('Chosen alphas (S40b) -> r24:', a24, '| r30:', a30)

Xz_te_r24 = build_Xz_te_with_recency(a24[0], a24[1])
z_te_r24 = (Xz_te_r24 @ coef) + intercept
p_meta_r24 = sigmoid(z_te_r24).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p_meta_r24}).to_csv('submission_s40b_meta_r24.csv', index=False)
print('Wrote submission_s40b_meta_r24.csv | mean', float(p_meta_r24.mean()))

Xz_te_r30 = build_Xz_te_with_recency(a30[0], a30[1])
z_te_r30 = (Xz_te_r30 @ coef) + intercept
p_meta_r30 = sigmoid(z_te_r30).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p_meta_r30}).to_csv('submission_s40b_meta_r30.csv', index=False)
print('Wrote submission_s40b_meta_r30.csv | mean', float(p_meta_r30.mean()))

def logit_avg(paths, out):
    zs=[to_logit(pd.read_csv(p)[target_col].values.astype(np.float64)) for p in paths]
    z=np.mean(zs,axis=0); p=sigmoid(z).astype(np.float32)
    pd.DataFrame({id_col: ids, target_col: p}).to_csv(out, index=False)
    print('Wrote', out, '| mean', float(p.mean()))

logit_avg(['submission_s40b_meta_gamma.csv','submission_s40b_meta_r24.csv'], 'submission_s40b_meta_hedge2.csv')
logit_avg(['submission_s40b_meta_gamma.csv','submission_s40b_meta_r24.csv','submission_s40b_meta_r30.csv'], 'submission_s40b_meta_hedge3.csv')

print('Note: not auto-promoting S40b; compare hedges vs S39 hedge before promoting.')

In [1]:
# S41-LGBM-OOF: True forward-chaining LightGBM stacker (no test-only features) + recency hedges + bias portfolio
import os, time, json, math
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

t0 = time.time()
print('S41-LGBM-OOF: starting true time-aware OOF LightGBM stacker ...')

id_col = 'request_id'; target_col = 'requester_received_pizza'
train = pd.read_json('train.json'); test = pd.read_json('test.json')
y = train[target_col].astype(int).values

def to_logit(p, eps=1e-6):
    p = np.clip(np.asarray(p, dtype=np.float64), eps, 1-eps)
    return np.log(p/(1-p))
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

# 1) Time ordering and 6 contiguous blocks
ts_col = 'unix_timestamp_of_request' if 'unix_timestamp_of_request' in train.columns else 'unix_timestamp_of_request_utc'
order = np.argsort(train[ts_col].values)
k = 6
blocks = np.array_split(order, k)
ntr = len(train)
nte = len(test)

# 2) Load base OOF/test probabilities for selected bases; convert to logits
base_specs = [
    ('LR_nosub', 'oof_lr_time_nosub_meta.npy', 'test_lr_time_nosub_meta.npy'),
    ('Dense_v1', 'oof_xgb_dense_time.npy', 'test_xgb_dense_time.npy'),
    ('Meta',    'oof_xgb_meta_time.npy',  'test_xgb_meta_time.npy'),
    ('MiniLM',  'oof_xgb_emb_meta_time.npy', 'test_xgb_emb_meta_time.npy'),
    ('MPNet',   'oof_xgb_emb_mpnet_time.npy', 'test_xgb_emb_mpnet_time.npy'),
    ('E5_meta', 'oof_xgb_e5_meta_time.npy', 'test_xgb_e5_meta_time.npy'),
    # Optional text+meta base if available
    ('CatBoost_v2', 'oof_catboost_textmeta_v2.npy', 'test_catboost_textmeta_v2.npy'),
]

base_raw = []  # list of (name, oof_prob, te_prob)
for name, oof_fp, te_fp in base_specs:
    if (not os.path.exists(oof_fp)) or (not os.path.exists(te_fp)):
        continue
    oof = np.load(oof_fp); tep = np.load(te_fp)
    if oof.ndim>1: oof=oof.ravel()
    if tep.ndim>1: tep=tep.ravel()
    if len(oof) != ntr:
        continue
    base_raw.append((name, oof.astype(np.float64), tep.astype(np.float64)))

if not base_raw:
    raise RuntimeError('No base OOF/Test pairs found. Aborting S41-LGBM-OOF.')

# 2b) Per-base last-block AUC and pruning: keep bases with AUC_last >= 0.60; optionally include CatBoost_v2 if >=0.65
last_block_idx = blocks[4]  # block 5 indices
def safe_auc(y_true, p):
    try:
        return float(roc_auc_score(y_true, p))
    except Exception:
        return float('nan')

rows = []
for (name, oof, te) in base_raw:
    auc_last = safe_auc(y[last_block_idx], oof[last_block_idx])
    rows.append((name, auc_last))
rows_sorted = sorted(rows, key=lambda t: (-(t[1] if not np.isnan(t[1]) else -1)))
print('Base last-block AUCs (desc):', rows_sorted)

# Basic keep by threshold 0.60
filtered = [(bn, o, t) for (bn, o, t) in base_raw if safe_auc(y[last_block_idx], o[last_block_idx]) >= 0.60]

# If CatBoost_v2 present but <0.65, drop it
filtered2 = []
for (bn, o, t) in filtered:
    if bn == 'CatBoost_v2':
        auc_n = safe_auc(y[last_block_idx], o[last_block_idx])
        if auc_n < 0.65:
            continue
    filtered2.append((bn, o, t))
filtered = filtered2

if len(filtered) > 7:
    keep_names = set([nm for (nm, _) in rows_sorted[:7]])
    filtered = [tpl for tpl in filtered if tpl[0] in keep_names]
if len(filtered) < 4:
    keep_names = set([nm for (nm, _) in rows_sorted[:6]])
    filtered = [tpl for tpl in base_raw if tpl[0] in keep_names]

names = [bn for (bn,_,_) in filtered]
Z_oof_list = [to_logit(o) for (_,o,_) in filtered]
Z_te_list  = [to_logit(t) for (_,_,t) in filtered]
print('Included bases after pruning:', names)
X_oof_b = np.column_stack(Z_oof_list).astype(np.float64)
X_te_b  = np.column_stack(Z_te_list).astype(np.float64)

# 3) Build meta features: log1p_text_len, account_age_days, time_rank01 (computed across train+test)
def get_text(df):
    title = df.get('request_title', pd.Series(['']*len(df))).fillna('').astype(str)
    body = df.get('request_text_edit_aware', df.get('request_text', pd.Series(['']*len(df)))).fillna('').astype(str)
    return title + '\n' + body
tr_txt = get_text(train); te_txt = get_text(test)
tr_len = tr_txt.str.len().values.astype(np.float64); te_len = te_txt.str.len().values.astype(np.float64)
tr_log1p = np.log1p(tr_len); te_log1p = np.log1p(te_len)

def get_numeric_col(df, col, length):
    if col in df.columns:
        s = pd.to_numeric(df[col], errors='coerce')
    else:
        s = pd.Series([np.nan]*length)
    s = s.fillna(0.0).astype(np.float64)
    return s.values

tr_age = get_numeric_col(train, 'requester_account_age_in_days_at_request', ntr)
te_age = get_numeric_col(test, 'requester_account_age_in_days_at_request', nte)

ts_tr = train[ts_col].values.astype(np.int64); ts_te = (test[ts_col].values.astype(np.int64) if ts_col in test.columns else np.zeros(nte, np.int64))
all_ts = np.concatenate([ts_tr, ts_te])
ord_all = np.argsort(all_ts)
rank_all = np.empty_like(ord_all)
rank_all[ord_all] = np.arange(len(all_ts))
rank01_all = rank_all.astype(np.float64) / max(1, (len(all_ts)-1))
tr_rank = rank01_all[:ntr]; te_rank = rank01_all[ntr:]

X_oof = np.column_stack([X_oof_b, tr_log1p, tr_age, tr_rank]).astype(np.float64)
X_te  = np.column_stack([X_te_b, te_log1p, te_age, te_rank]).astype(np.float64)
feat_cols = names + ['log1p_text_len','account_age_days','time_rank01']
nb = len(names)
print('Final meta columns:', feat_cols)

# 4) True forward-chaining OOF for meta (validate on blocks 2..5)
val_blocks = [2,3,4,5]
oof_pred = np.full(ntr, np.nan, dtype=np.float64)
gamma = 0.995

params = dict(
    learning_rate=0.03,
    num_leaves=31,
    max_depth=5,
    min_data_in_leaf=80,
    feature_fraction=1.0,
    bagging_fraction=0.8,
    bagging_freq=1,
    lambda_l2=5.0,
    min_gain_to_split=0.001,
    n_estimators=1200,
    objective='binary',
    random_state=42
)

for vb in val_blocks:
    tr_idx = np.concatenate([blocks[i-1] for i in range(1, vb)])  # blocks < vb
    va_idx = blocks[vb-1]  # block vb
    gbm = lgb.LGBMClassifier(**params)
    gbm.fit(
        X_oof[tr_idx], y[tr_idx],
        eval_set=[(X_oof[va_idx], y[va_idx])],
        eval_metric='auc',
        callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(50)]
    )
    oof_pred[va_idx] = gbm.predict_proba(X_oof[va_idx], num_iteration=gbm.best_iteration_)[:,1]
    print(f'VB={vb} done | va_size={len(va_idx)} | elapsed={time.time()-t0:.1f}s | best_iter={gbm.best_iteration_}')

# Report AUC_last (block 5) and gamma-weighted AUC over blocks 2..5 (double weight for block 5)
mask_oof = np.isfinite(oof_pred)
assert mask_oof.sum() == sum(len(blocks[i-1]) for i in val_blocks)
auc_last = roc_auc_score(y[blocks[4-1]], oof_pred[blocks[4-1]])  # block 5 -> index 4
W = np.zeros(ntr, dtype=np.float64)
for bi in range(1, k):
    age = (k - 1) - bi
    w_block = (gamma ** age)
    if bi == 5:  # double weight for last validated block
        w_block *= 2.0
    W[blocks[bi]] = w_block
auc_full_gamma = roc_auc_score(y[mask_oof], oof_pred[mask_oof], sample_weight=W[mask_oof])
print(f'AUC_last (block5)={auc_last:.5f} | AUC_full_gamma(b2..b5)={auc_full_gamma:.5f}')

# 5) Final meta fit: train on blocks 1..4, early stop on block 5
tr_final_idx = np.concatenate([blocks[i] for i in range(0,4)])  # 0..3
va_final_idx = blocks[4]  # block 5
final_model = lgb.LGBMClassifier(**params)
final_model.fit(
    X_oof[tr_final_idx], y[tr_final_idx],
    eval_set=[(X_oof[va_final_idx], y[va_final_idx])],
    eval_metric='auc',
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(50)]
)
print('Final meta trained | best_iter=', final_model.best_iteration_)

# 6) meta_gamma prediction
p_gamma = final_model.predict_proba(X_te, num_iteration=final_model.best_iteration_)[:,1].astype(np.float32)
pd.DataFrame({id_col: test[id_col].values, target_col: p_gamma}).to_csv('submission_s41lgb_meta_gamma.csv', index=False)
print('Wrote submission_s41lgb_meta_gamma.csv | mean', float(p_gamma.mean()))

# 7) Safe recency injection at test only: build r_low and r_high feature variants, then reuse trained meta
recent_map = {
    'LR_nosub': [
        'test_lr_time_nosub_meta_recent35.npy',
        'test_lr_time_nosub_meta_recent45.npy',
    ],
    'MiniLM': [
        'test_xgb_emb_meta_time_recent35.npy',
        'test_xgb_emb_meta_time_recent45.npy',
    ],
    'MPNet': [
        'test_xgb_emb_mpnet_time_recent35.npy',
        'test_xgb_emb_mpnet_time_recent45.npy',
    ],
}
name_to_col = {n:i for i,n in enumerate(names)}

def load_recent_avg_logit(files):
    arrs = []
    for fp in files:
        if os.path.exists(fp):
            a = np.load(fp)
            if a.ndim>1: a=a.ravel()
            arrs.append(to_logit(a))
    if not arrs:
        return None
    return np.mean(arrs, axis=0).astype(np.float64)

def apply_recency_to_Xte(X_base, alphas):
    # alphas: dict base_name -> alpha, interpolate in logit space
    Xr = X_base.copy()
    for bname, a in alphas.items():
        if a <= 0: continue
        if bname not in name_to_col: continue
        rec = load_recent_avg_logit(recent_map.get(bname, []))
        if rec is None: continue
        j = name_to_col[bname]
        z_full = X_base[:, j]
        z_recent = rec
        Xr[:, j] = (1.0 - a)*z_full + a*z_recent
    return Xr

# r_low and r_high per expert advice (no training-time recency)
alphas_low = {'LR_nosub': 0.00, 'MiniLM': 0.15, 'MPNet': 0.20}
alphas_high = {'LR_nosub': 0.05, 'MiniLM': 0.25, 'MPNet': 0.30}

X_te_low = apply_recency_to_Xte(X_te, alphas_low)
X_te_high = apply_recency_to_Xte(X_te, alphas_high)
p_low = final_model.predict_proba(X_te_low, num_iteration=final_model.best_iteration_)[:,1].astype(np.float32)
p_high = final_model.predict_proba(X_te_high, num_iteration=final_model.best_iteration_)[:,1].astype(np.float32)
pd.DataFrame({id_col: test[id_col].values, target_col: p_low}).to_csv('submission_s41lgb_meta_r_low.csv', index=False)
pd.DataFrame({id_col: test[id_col].values, target_col: p_high}).to_csv('submission_s41lgb_meta_r_high.csv', index=False)
print('Wrote r_low/r_high | means ->', float(p_low.mean()), float(p_high.mean()))

# 8) 3-way logit hedge: gamma, r_low, r_high
def logit_clip(p, eps=1e-6):
    p = np.clip(p.astype(np.float64), eps, 1-eps)
    return np.log(p/(1-p))
z_g = logit_clip(p_gamma); z_l = logit_clip(p_low); z_h = logit_clip(p_high)
p_hedge3 = sigmoid((z_g + z_l + z_h)/3.0).astype(np.float32)
pd.DataFrame({id_col: test[id_col].values, target_col: p_hedge3}).to_csv('submission_s41lgb_meta_hedge3.csv', index=False)
print('Wrote submission_s41lgb_meta_hedge3.csv | mean', float(p_hedge3.mean()))

# 9) Bias portfolio: means 0.30 (promote), 0.32, 0.28 using monotone logit shift
def bias_to_mean(probs, target, tol=1e-6, it=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(it):
        mid = 0.5*(lo+hi); m = sigmoid(z+mid).mean()
        if abs(m - target) < tol: return mid
        if m < target: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

for tm in [0.30, 0.32, 0.28]:
    b = bias_to_mean(p_hedge3, tm)
    pm = sigmoid(logit_clip(p_hedge3) + b).astype(np.float32)
    outp = f'submission_s41lgb_meta_hedge3_m{int(round(tm*100)):03d}.csv'
    pd.DataFrame({id_col: test[id_col].values, target_col: pm}).to_csv(outp, index=False)
    print(f'Wrote {outp} | mean={pm.mean():.6f} | bias={b:.4f}')
    if abs(tm - 0.30) < 1e-9:
        pd.DataFrame({id_col: test[id_col].values, target_col: pm}).to_csv('submission.csv', index=False)
        print('PROMOTED: submission.csv <-', outp)

print(f'S41-LGBM-OOF done in {time.time()-t0:.1f}s')

S41-LGBM-OOF: starting true time-aware OOF LightGBM stacker ...
Base last-block AUCs (desc): [('CatBoost_v2', 0.650664850639457), ('MiniLM', 0.6444665035320191), ('Dense_v1', 0.6396417193776259), ('MPNet', 0.6346091693984025), ('LR_nosub', 0.6235744955907474), ('Meta', 0.621023592963664), ('E5_meta', 0.6164642873632208)]
Included bases after pruning: ['LR_nosub', 'Dense_v1', 'Meta', 'MiniLM', 'MPNet', 'E5_meta', 'CatBoost_v2']
Final meta columns: ['LR_nosub', 'Dense_v1', 'Meta', 'MiniLM', 'MPNet', 'E5_meta', 'CatBoost_v2', 'log1p_text_len', 'account_age_days', 'time_rank01']
[LightGBM] [Warning] min_data_in_leaf is set=80, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=80
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_gain_to_split is set=0.001, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0.001
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[50]	valid_0's auc: 0.703887	valid_0's binary_logloss: 0.543729
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

/app/.pip-target/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/app/.pip-target/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/app/.pip-target/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/app/.pip-target/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[100]	valid_0's auc: 0.632693	valid_0's binary_logloss: 0.546232
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spl

/app/.pip-target/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/app/.pip-target/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/app/.pip-target/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# S41d-bias: Monotonic logit bias shift to target mean=0.40 and 50/50 hedge
import numpy as np, pandas as pd, math, os, sys, time

def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))

def logit(p):
    p = np.clip(p, 1e-6, 1-1e-6)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit(probs)
    # Bisection on bias b: f(b)=mean(sigmoid(z+b)) - target
    lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi)
        m = sigmoid(z+mid).mean()
        if abs(m - target_mean) < tol:
            return mid
        if m < target_mean:
            lo = mid
        else:
            hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, bias):
    return sigmoid(logit(probs) + bias)

def load_sub(path):
    df = pd.read_csv(path)
    # Normalize column names
    cols = [c.lower() for c in df.columns]
    if 'request_id' in cols:
        id_col = df.columns[cols.index('request_id')]
    elif 'id' in cols:
        id_col = df.columns[cols.index('id')]
    else:
        raise ValueError('No id/request_id column found in ' + path)
    prob_col = [c for c in df.columns if c != id_col][0]
    return df[[id_col, prob_col]].rename(columns={id_col:'request_id', prob_col:'requester_received_pizza'})

def save_sub(df, path):
    out = df.copy()
    out.columns = ['request_id','requester_received_pizza']
    out.to_csv(path, index=False)
    print(f'Wrote {os.path.basename(path)} | mean={out.requester_received_pizza.mean():.6f}')

t0 = time.time()
gamma_path = 'submission_s41_meta_gamma.csv'
r24_path = 'submission_s41_meta_r24.csv'
assert os.path.exists(gamma_path), 'Missing ' + gamma_path
assert os.path.exists(r24_path), 'Missing ' + r24_path
sg = load_sub(gamma_path)
sr = load_sub(r24_path)
assert np.all(sg.request_id.values == sr.request_id.values), 'Mismatched ids between gamma and r24'

target_mean = 0.40
print(f'Original means: gamma={sg.requester_received_pizza.mean():.6f} | r24={sr.requester_received_pizza.mean():.6f}')
b_g = find_bias_for_target_mean(sg.requester_received_pizza.values, target_mean)
b_r = find_bias_for_target_mean(sr.requester_received_pizza.values, target_mean)
pg = apply_bias(sg.requester_received_pizza.values, b_g)
pr = apply_bias(sr.requester_received_pizza.values, b_r)
sg_m = pd.DataFrame({'request_id': sg.request_id.values, 'requester_received_pizza': pg})
sr_m = pd.DataFrame({'request_id': sr.request_id.values, 'requester_received_pizza': pr})
save_sub(sg_m, 'submission_s41_meta_gamma_m040.csv')
save_sub(sr_m, 'submission_s41_meta_r24_m040.csv')

# Logit-hedge 50/50 after shifting each to mean 0.40
lg = logit(pg)
lr_ = logit(pr)
hedge_logits = 0.5*(lg + lr_)
ph = sigmoid(hedge_logits)
sh_m = pd.DataFrame({'request_id': sg.request_id.values, 'requester_received_pizza': ph})
save_sub(sh_m, 'submission_s41_meta_hedge_m040.csv')

# Promote hedge to submission.csv
save_sub(sh_m, 'submission.csv')
print(f'S41d-bias completed in {time.time()-t0:.2f}s')

In [ ]:
# S41d-bias-variants: Generate mean 0.38 and 0.42 hedged submissions
import numpy as np, pandas as pd, os, time

def sigmoid(x):
    return 1.0/(1.0+np.exp(-x))
def logit(p):
    p = np.log(np.clip(p,1e-6,1-1e-6)/(1-np.clip(p,1e-6,1-1e-6)))
def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit(probs)
    lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi)
        m = sigmoid(z+mid).mean()
        if abs(m - target_mean) < tol:
            return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)
def apply_bias(probs, bias):
    return sigmoid(logit(probs)+bias)
def load_sub(path):
    df = pd.read_csv(path)
    cols = [c.lower() for c in df.columns]
    if 'request_id' in cols:
        id_col = df.columns[cols.index('request_id')]
    elif 'id' in cols:
        id_col = df.columns[cols.index('id')]
    else:
        raise ValueError('No id/request_id column found in ' + path)
    prob_col = [c for c in df.columns if c != id_col][0]
    return df[[id_col, prob_col]].rename(columns={id_col:'request_id', prob_col:'requester_received_pizza'})
def save_sub(df, path):
    out = df.copy()
    out.columns = ['request_id','requester_received_pizza']
    out.to_csv(path, index=False)
    print(f'Wrote {os.path.basename(path)} | mean={out.requester_received_pizza.mean():.6f}')

gamma_path = 'submission_s41_meta_gamma.csv'
r24_path = 'submission_s41_meta_r24.csv'
sg = load_sub(gamma_path)
sr = load_sub(r24_path)
assert np.all(sg.request_id.values == sr.request_id.values)

for tm in [0.38, 0.42]:
    b_g = find_bias_for_target_mean(sg.requester_received_pizza.values, tm)
    b_r = find_bias_for_target_mean(sr.requester_received_pizza.values, tm)
    pg = apply_bias(sg.requester_received_pizza.values, b_g)
    pr = apply_bias(sr.requester_received_pizza.values, b_r)
    lg, lr_ = logit(pg), logit(pr)
    ph = sigmoid(0.5*(lg+lr_))
    dfh = pd.DataFrame({'request_id': sg.request_id.values, 'requester_received_pizza': ph})
    save_sub(dfh, f'submission_s41_meta_hedge_m{int(tm*100):03d}.csv')

# Promote 0.42 variant as next submission
df42 = pd.read_csv('submission_s41_meta_hedge_m042.csv')
df42.columns = ['request_id','requester_received_pizza']
df42.to_csv('submission.csv', index=False)
print(f'Promoted submission.csv | mean={df42.requester_received_pizza.mean():.6f}')

In [ ]:
# S41d-bias-variants (fixed): Use existing bias-shift helpers from Cell 7 to make 0.38 and 0.42 hedges
import numpy as np, pandas as pd, os, time

t0 = time.time()
gamma_path = 'submission_s41_meta_gamma.csv'
r24_path = 'submission_s41_meta_r24.csv'
assert os.path.exists(gamma_path) and os.path.exists(r24_path), 'Missing gamma/r24 files'
sg = load_sub(gamma_path)
sr = load_sub(r24_path)
assert np.all(sg.request_id.values == sr.request_id.values), 'ID mismatch'

for tm in [0.38, 0.42]:
    b_g = find_bias_for_target_mean(sg.requester_received_pizza.values, tm)
    b_r = find_bias_for_target_mean(sr.requester_received_pizza.values, tm)
    pg = apply_bias(sg.requester_received_pizza.values, b_g)
    pr = apply_bias(sr.requester_received_pizza.values, b_r)
    lg, lr_ = logit(pg), logit(pr)
    ph = sigmoid(0.5*(lg + lr_))
    dfh = pd.DataFrame({'request_id': sg.request_id.values, 'requester_received_pizza': ph})
    out_path = f'submission_s41_meta_hedge_m{int(round(tm*100)):03d}.csv'
    save_sub(dfh, out_path)

# Promote 0.42 variant
df42 = pd.read_csv('submission_s41_meta_hedge_m042.csv')
df42.columns = ['request_id','requester_received_pizza']
df42.to_csv('submission.csv', index=False)
print(f'Promoted submission.csv | mean={df42.requester_received_pizza.mean():.6f} | took {time.time()-t0:.2f}s')

In [ ]:
# Promote 0.38-mean hedge to submission.csv and report mean
import pandas as pd
df38 = pd.read_csv('submission_s41_meta_hedge_m038.csv')
df38.columns = ['request_id','requester_received_pizza']
df38.to_csv('submission.csv', index=False)
print(f'Promoted submission.csv | mean={df38.requester_received_pizza.mean():.6f}')

In [ ]:
# S41c: Platt calibration on S41 outputs to fix underconfidence and re-hedge
import numpy as np, pandas as pd, time
from sklearn.linear_model import LogisticRegression
from scipy.special import expit

t0 = time.time()
print('S41c: starting Platt calibration...')

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def sigmoid(x):
    return expit(x)

# Guard: ensure prior cell ran
assert 'oof_pred' in globals() and 'y_tr' in globals() and 'W_tr' in globals(), 'S41 artifacts missing'

# Fit Platt on logit(oof_pred) -> y_tr with sample weights
z_tr = logit_clip(oof_pred).reshape(-1, 1)
platt = LogisticRegression(C=10.0, solver='lbfgs', max_iter=1000, class_weight=None, random_state=42)
platt.fit(z_tr, y_tr, sample_weight=W_tr)
a = float(platt.coef_.ravel()[0])
b = float(platt.intercept_.ravel()[0])
print(f'Platt params: a={a:.4f}, b={b:.4f}')

# Load S41 submissions and apply calibration
sub_g = pd.read_csv('submission_s41_meta_gamma.csv')
sub_r24_path = 'submission_s41_meta_r24.csv'
sub_r20_path = 'submission_s41_meta_r20.csv'
sub_r_path = sub_r24_path if os.path.exists(sub_r24_path) else sub_r20_path
sub_r = pd.read_csv(sub_r_path)

def apply_platt_to_series(p):
    z = logit_clip(p.values)
    return pd.Series(sigmoid(a*z + b), index=p.index)

sub_g['requester_received_pizza'] = apply_platt_to_series(sub_g['requester_received_pizza'])
sub_r['requester_received_pizza'] = apply_platt_to_series(sub_r['requester_received_pizza'])

sub_g.to_csv('submission_s41c_meta_gamma.csv', index=False)
sub_r.to_csv('submission_s41c_meta_r.csv', index=False)
print(f'Calibrated means -> gamma: {sub_g.requester_received_pizza.mean():.6f} | r: {sub_r.requester_received_pizza.mean():.6f}')

# 2-way logit hedge after calibration
p1 = sub_g.requester_received_pizza.values
p2 = sub_r.requester_received_pizza.values
lz = 0.5*logit_clip(p1) + 0.5*logit_clip(p2)
p_hedge = sigmoid(lz)
sub_h = pd.DataFrame({'id': sub_g.id, 'requester_received_pizza': p_hedge})
sub_h.to_csv('submission_s41c_meta_hedge2.csv', index=False)
m_h = sub_h.requester_received_pizza.mean()
print(f'Wrote submission_s41c_meta_hedge2.csv | mean {m_h:.6f}')

# Promotion guardrail: AUC unchanged by monotone calibration; reuse S41 auc in memory
auc_ok = ('auc' in globals()) and (auc >= 0.692)
mean_ok = 0.39 <= m_h <= 0.43
if auc_ok and mean_ok:
    sub_h.to_csv('submission.csv', index=False)
    print('PROMOTED: submission.csv <- submission_s41c_meta_hedge2.csv')
else:
    print(f'Not promoting S41c; auc_ok={auc_ok}, mean_ok={mean_ok}')

print(f'S41c done in {time.time()-t0:.2f}s')

In [ ]:
# S41d: Monotonic bias correction to target submission mean ~0.40 without changing ranking
import numpy as np, pandas as pd, time, os
from scipy.special import expit

print('S41d: starting bias correction to target mean ~0.40')

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def shift_to_target_mean(p, target=0.40):
    z = logit_clip(p)
    # bisection on delta so that mean(sigmoid(z+delta)) ~= target
    lo, hi = -10.0, 10.0
    for _ in range(50):
        mid = 0.5*(lo+hi)
        m = expit(z + mid).mean()
        if m < target:
            lo = mid
        else:
            hi = mid
    delta = 0.5*(lo+hi)
    p_new = expit(z + delta)
    return p_new, float(delta), float(p_new.mean())

# Load S41 base submissions
sub_g = pd.read_csv('submission_s41_meta_gamma.csv')
sub_r_path = 'submission_s41_meta_r24.csv' if os.path.exists('submission_s41_meta_r24.csv') else 'submission_s41_meta_r20.csv'
sub_r = pd.read_csv(sub_r_path)

# Target mean window per guardrail
target_mean = 0.40
p_g_new, d_g, m_g = shift_to_target_mean(sub_g.requester_received_pizza.values, target=target_mean)
p_r_new, d_r, m_r = shift_to_target_mean(sub_r.requester_received_pizza.values, target=target_mean)
print(f'Applied deltas -> gamma: {d_g:.4f} (mean {m_g:.6f}), recent: {d_r:.4f} (mean {m_r:.6f})')

sub_g_cal = pd.DataFrame({'id': sub_g.id, 'requester_received_pizza': p_g_new})
sub_r_cal = pd.DataFrame({'id': sub_r.id, 'requester_received_pizza': p_r_new})
sub_g_cal.to_csv('submission_s41d_meta_gamma.csv', index=False)
sub_r_cal.to_csv('submission_s41d_meta_r.csv', index=False)
print(f'Wrote submission_s41d_meta_gamma.csv | mean {m_g:.6f}')
print(f'Wrote submission_s41d_meta_r.csv | mean {m_r:.6f}')

# 2-way logit-average hedge after bias correction
lz1 = logit_clip(sub_g_cal.requester_received_pizza.values)
lz2 = logit_clip(sub_r_cal.requester_received_pizza.values)
p_hedge = expit(0.5*lz1 + 0.5*lz2)
sub_h = pd.DataFrame({'id': sub_g_cal.id, 'requester_received_pizza': p_hedge})
sub_h.to_csv('submission_s41d_meta_hedge2.csv', index=False)
m_h = sub_h.requester_received_pizza.mean()
print(f'Wrote submission_s41d_meta_hedge2.csv | mean {m_h:.6f}')

# Promote if AUC from S41 was sufficient and mean within window
auc_ok = ('auc' in globals()) and (auc >= 0.692)
mean_ok = 0.39 <= m_h <= 0.43
if auc_ok and mean_ok:
    sub_h.to_csv('submission.csv', index=False)
    print('PROMOTED: submission.csv <- submission_s41d_meta_hedge2.csv')
else:
    print(f'Not promoting S41d; auc_ok={auc_ok}, mean_ok={mean_ok}')

In [ ]:
# S41e: Validate submission.csv format for Kaggle
import pandas as pd, numpy as np
import json, os
print('Validating submission.csv ...')
sub = pd.read_csv('submission.csv')
print('Shape:', sub.shape)
print('Columns:', list(sub.columns))
print(sub.head(5))
n_na = sub.isna().sum().to_dict()
print('NA per column:', n_na)
print('requester_received_pizza stats -> min/max/mean:', float(sub['requester_received_pizza'].min()), float(sub['requester_received_pizza'].max()), float(sub['requester_received_pizza'].mean()))
print('Dtypes:', sub.dtypes.to_dict())
print('Unique id count:', sub['id'].nunique())
print('Any probs out of bounds:', bool(((sub['requester_received_pizza'] < 0) | (sub['requester_received_pizza'] > 1)).any()))
with open('test.json', 'r') as f:
    te_json = json.load(f)
expected_n = len(te_json)
print('Expected test rows:', expected_n)
if sub.shape[0] != expected_n:
    print('ERROR: Row count mismatch')
else:
    # Check id match set
    te_ids = pd.Series([x.get('id') for x in te_json])
    set_diff1 = set(sub['id']) - set(te_ids)
    set_diff2 = set(te_ids) - set(sub['id'])
    print('ID diff sizes -> sub-not-in-test:', len(set_diff1), ' test-not-in-sub:', len(set_diff2))
    if len(set_diff1) or len(set_diff2):
        print('Sample differences (up to 5):', list(sorted(list(set_diff1))[:5]), list(sorted(list(set_diff2))[:5]))
print('Validation done.')

In [ ]:
# S41f: Fix submission IDs from test.json
import json, pandas as pd, numpy as np, os
print('S41f: inspecting test.json keys to fix submission IDs...')
with open('test.json', 'r') as f:
    te_json = json.load(f)
n_te = len(te_json)
print('Test rows:', n_te)

candidates = ['id', 'request_id', 'post_id', 'name']
counts = {}
for k in candidates:
    vals = [x.get(k) for x in te_json]
    counts[k] = sum(v is not None for v in vals)
print('Non-null counts per key:', counts)

# choose first key with full coverage
chosen = None
for k in candidates:
    if counts.get(k, 0) == n_te:
        chosen = k
        break
if chosen is None:
    # fallback: prefer request_id if mostly present
    chosen = max(counts, key=lambda k: counts[k])
print('Chosen key for IDs:', chosen)

ids = [x.get(chosen) for x in te_json]
# ensure no None remains; if any, fill with sequential indices as last resort
if any(v is None for v in ids):
    print('Warning: some IDs missing; filling with index-based placeholders')
    ids = [v if v is not None else i for i, v in enumerate(ids)]

# Load best probs from S41d hedge (already promoted earlier)
sub_path = 'submission_s41d_meta_hedge2.csv' if os.path.exists('submission_s41d_meta_hedge2.csv') else 'submission_s41_meta_hedge2.csv'
sub = pd.read_csv(sub_path)
print('Loaded probs from', sub_path, 'shape', sub.shape, 'mean', float(sub['requester_received_pizza'].mean()))

# Overwrite id column with extracted IDs in same order as test.json
sub_fixed = pd.DataFrame({'id': ids, 'requester_received_pizza': sub['requester_received_pizza'].values})
print('Fixed NA in id:', int(sub_fixed['id'].isna().sum()))

# Save fixed submission and promote
sub_fixed.to_csv('submission_fixed.csv', index=False)
sub_fixed.to_csv('submission.csv', index=False)
print('Wrote submission_fixed.csv and promoted to submission.csv')
print('Head:')
print(sub_fixed.head(3))

In [ ]:
# S41g: Ensure submission has correct header 'request_id' per competition format
import pandas as pd, json, os
print('S41g: rewriting submission.csv with request_id header')
sub = pd.read_csv('submission.csv')
cols = list(sub.columns)
print('Current columns:', cols)
if 'request_id' not in sub.columns and 'id' in sub.columns:
    sub = sub.rename(columns={'id': 'request_id'})
    print('Renamed id -> request_id')
sub = sub[['request_id', 'requester_received_pizza']]
sub.to_csv('submission.csv', index=False)
print('Wrote submission.csv with columns:', list(sub.columns))

# Quick validation against test.json request_id keys
with open('test.json', 'r') as f:
    te_json = json.load(f)
test_ids = [x.get('request_id') for x in te_json]
print('Rows:', len(sub), ' expected:', len(test_ids))
print('NA in request_id:', int(sub['request_id'].isna().sum()))
print('Mean prob:', float(sub['requester_received_pizza'].mean()))
print('Header OK:', list(sub.columns) == ['request_id', 'requester_received_pizza'])

In [ ]:
# S41-recent: Generate required recent35/45 test-only logits for LR_nosub and MiniLM
import numpy as np, pandas as pd, os, time
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

try:
    import xgboost as xgb
    HAS_XGB = True
except Exception:
    HAS_XGB = False

t0 = time.time()
id_col = 'request_id'
target_col = 'requester_received_pizza'
ts_keys = ['unix_timestamp_of_request', 'unix_timestamp_of_request_utc']

train = pd.read_json('train.json')
test = pd.read_json('test.json')
y = train[target_col].astype(int).values
n = len(train)

# unix timestamp (with fallback)
if ts_keys[0] in train.columns:
    ts = train[ts_keys[0]].values
else:
    ts = train[ts_keys[1]].values

# Time order and recent slices
order = np.argsort(ts)  # ascending
cut35 = int(np.floor(0.65 * n))  # last 35%
cut45 = int(np.floor(0.55 * n))  # last 45%
idx_r35 = order[cut35:]
idx_r45 = order[cut45:]
print(f'n={n} | r35 size={len(idx_r35)} | r45 size={len(idx_r45)}')

# Cached features
meta_tr = np.load('meta_v1_tr.npy')  # (n, m_meta)
meta_te = np.load('meta_v1_te.npy')
emb_mn_tr = np.load('emb_minilm_tr.npy')  # (n, d)
emb_mn_te = np.load('emb_minilm_te.npy')

# Text concat (title + body)
def get_text(df: pd.DataFrame) -> pd.Series:
    title = df.get('request_title', pd.Series(['']*len(df))).fillna('').astype(str)
    body = df.get('request_text_edit_aware', df.get('request_text', pd.Series(['']*len(df)))).fillna('').astype(str)
    return title + ' ' + body

txt_tr = get_text(train)
txt_te = get_text(test)

# 1) LR_nosub recent35/45 -> test_lr_time_nosub_meta_recentXX.npy
tfidf_params = dict(
    analyzer='word',
    ngram_range=(1, 2),
    min_df=3,
    max_features=100000,
    lowercase=True,
    strip_accents='unicode',
    sublinear_tf=True
)

def fit_predict_lr_recent(idx_recent, out_path):
    v = TfidfVectorizer(**tfidf_params)
    X_txt_tr = v.fit_transform(txt_tr.iloc[idx_recent])
    X_txt_te = v.transform(txt_te)
    X_tr = sparse.hstack([X_txt_tr, sparse.csr_matrix(meta_tr[idx_recent])], format='csr')
    X_te = sparse.hstack([X_txt_te, sparse.csr_matrix(meta_te)], format='csr')

    lr = LogisticRegression(
        penalty='l2',
        solver='liblinear',
        C=1.0,
        max_iter=2000,
        random_state=42
    )
    lr.fit(X_tr, y[idx_recent])
    p = lr.predict_proba(X_te)[:, 1].astype(np.float32)
    np.save(out_path, p)
    print(f'Wrote {out_path} | mean={float(p.mean()):.6f}')

fit_predict_lr_recent(idx_r35, 'test_lr_time_nosub_meta_recent35.npy')
fit_predict_lr_recent(idx_r45, 'test_lr_time_nosub_meta_recent45.npy')

# 2) MiniLM recent35/45 -> test_xgb_emb_meta_time_recentXX.npy
def fit_predict_minilm_recent(idx_recent, out_path):
    X_tr_slice = np.hstack([emb_mn_tr[idx_recent], meta_tr[idx_recent]])
    X_te = np.hstack([emb_mn_te, meta_te])

    if HAS_XGB:
        idx_recent_sorted = np.sort(idx_recent)
        m = len(idx_recent_sorted)
        split = int(np.floor(0.80 * m))
        tr_idx = idx_recent_sorted[:split]
        va_idx = idx_recent_sorted[split:]

        Xtr = np.hstack([emb_mn_tr[tr_idx], meta_tr[tr_idx]])
        Xva = np.hstack([emb_mn_tr[va_idx], meta_tr[va_idx]])
        ytr = y[tr_idx]
        yva = y[va_idx]

        dtr = xgb.DMatrix(Xtr, label=ytr)
        dva = xgb.DMatrix(Xva, label=yva)
        dte = xgb.DMatrix(X_te)

        params = {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'max_depth': 3,
            'eta': 0.05,
            'subsample': 0.9,
            'colsample_bytree': 0.9,
            'reg_lambda': 1.0,
            'reg_alpha': 0.0,
            'min_child_weight': 1.0,
            'tree_method': 'hist',
            'seed': 42
        }
        bst = xgb.train(params, dtr, num_boost_round=300, evals=[(dva, 'valid')], early_stopping_rounds=50, verbose_eval=False)
        p = bst.predict(dte, iteration_range=(0, bst.best_iteration + 1)).astype(np.float32)
    else:
        lr = LogisticRegression(penalty='l2', solver='lbfgs', C=1.0, max_iter=1000, random_state=42)
        lr.fit(X_tr_slice, y[idx_recent])
        p = lr.predict_proba(X_te)[:, 1].astype(np.float32)

    np.save(out_path, p)
    print(f'Wrote {out_path} | mean={float(p.mean()):.6f}')

fit_predict_minilm_recent(idx_r35, 'test_xgb_emb_meta_time_recent35.npy')
fit_predict_minilm_recent(idx_r45, 'test_xgb_emb_meta_time_recent45.npy')

print(f'Done generating 4 recent test-only files in {time.time()-t0:.1f}s')

In [ ]:
# S41r30: Build stronger recency variant (alpha=0.20) and 3-way hedges; also bias-shift to mean 0.40
import os, numpy as np, pandas as pd
from scipy.special import expit

# Guards: ensure S41-rev ran (clf and X_test exist)
assert 'clf' in globals() and 'X_test' in globals() and 'all_cols' in globals(), 'Run Cell 6 (S41-rev) first.'

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def try_load(fp):
    if os.path.exists(fp):
        arr = np.load(fp)
        if arr.ndim > 1: arr = arr.ravel()
        return arr
    return None

# Recent files map (same as S41)
recent_map = {
    'LR_nosub': [
        'test_lr_time_nosub_meta_recent35.npy',
        'test_lr_time_nosub_meta_recent45.npy',
    ],
    'MiniLM': [
        'test_xgb_emb_meta_time_recent35.npy',
        'test_xgb_emb_meta_time_recent45.npy',
    ],
}

col_to_idx = {c: i for i, c in enumerate(all_cols)}

def apply_recency(X_base, a_lr=0.20, a_mn=0.20):
    Xr = X_base.copy()
    # LR_nosub
    if 'LR_nosub' in col_to_idx and a_lr > 0:
        bi = col_to_idx['LR_nosub']
        z_full = X_base[:, bi]
        zs = [logit_clip(try_load(fp)) for fp in recent_map['LR_nosub'] if try_load(fp) is not None]
        if zs:
            z_recent = np.mean(zs, axis=0)
            Xr[:, bi] = (1.0 - a_lr)*z_full + a_lr*z_recent
    # MiniLM
    if 'MiniLM' in col_to_idx and a_mn > 0:
        bi = col_to_idx['MiniLM']
        z_full = X_base[:, bi]
        zs = [logit_clip(try_load(fp)) for fp in recent_map['MiniLM'] if try_load(fp) is not None]
        if zs:
            z_recent = np.mean(zs, axis=0)
            Xr[:, bi] = (1.0 - a_mn)*z_full + a_mn*z_recent
    return Xr

# 1) Build r30 with alpha=0.20
X_test_r30 = apply_recency(X_test, a_lr=0.20, a_mn=0.20)
p_r30 = clf.predict_proba(X_test_r30)[:, 1].astype(np.float32)
sub_r30 = pd.DataFrame({'request_id': pd.read_json('test.json')['request_id'].values, 'requester_received_pizza': p_r30})
sub_r30.to_csv('submission_s41_meta_r30.csv', index=False)
print('Wrote submission_s41_meta_r30.csv | mean', float(p_r30.mean()))

# 2) 3-way logit hedge (gamma, r24, r30)
sub_g = pd.read_csv('submission_s41_meta_gamma.csv')
sub_r24 = pd.read_csv('submission_s41_meta_r24.csv')
sub_r30 = pd.read_csv('submission_s41_meta_r30.csv')
assert np.all(sub_g.request_id.values == sub_r24.request_id.values) and np.all(sub_g.request_id.values == sub_r30.request_id.values), 'ID mismatch across submissions'
z_g = logit_clip(sub_g.requester_received_pizza.values)
z_r24 = logit_clip(sub_r24.requester_received_pizza.values)
z_r30 = logit_clip(sub_r30.requester_received_pizza.values)
z3 = (z_g + z_r24 + z_r30) / 3.0
p3 = expit(z3).astype(np.float32)
sub_h3 = pd.DataFrame({'request_id': sub_g.request_id, 'requester_received_pizza': p3})
sub_h3.to_csv('submission_s41_meta_hedge3.csv', index=False)
print('Wrote submission_s41_meta_hedge3.csv | mean', float(p3.mean()))

# 3) Bias-shift gamma, r24, r30 individually to mean 0.40, then 3-way hedge
def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs)
    lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi)
        m = expit(z+mid).mean()
        if abs(m - target_mean) < tol:
            return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, bias):
    return expit(logit_clip(probs) + bias)

tm = 0.40
b_g = find_bias_for_target_mean(sub_g.requester_received_pizza.values, tm)
b_r24 = find_bias_for_target_mean(sub_r24.requester_received_pizza.values, tm)
b_r30 = find_bias_for_target_mean(sub_r30.requester_received_pizza.values, tm)
pg = apply_bias(sub_g.requester_received_pizza.values, b_g)
pr24 = apply_bias(sub_r24.requester_received_pizza.values, b_r24)
pr30 = apply_bias(sub_r30.requester_received_pizza.values, b_r30)
zg, zr24, zr30 = logit_clip(pg), logit_clip(pr24), logit_clip(pr30)
p3m = expit((zg+zr24+zr30)/3.0).astype(np.float32)
sub_h3m = pd.DataFrame({'request_id': sub_g.request_id, 'requester_received_pizza': p3m})
sub_h3m.to_csv('submission_s41_meta_hedge3_m040.csv', index=False)
print('Wrote submission_s41_meta_hedge3_m040.csv | mean', float(p3m.mean()))

# Promote mean-0.40 3-way hedge to submission.csv
sub_h3m.to_csv('submission.csv', index=False)
print('Promoted submission.csv (3-way, mean~0.40)')

In [ ]:
# S41-recent-mpnet: Generate recent35/45 test-only logits for MPNet+meta
import numpy as np, pandas as pd, os, time
from sklearn.linear_model import LogisticRegression

try:
    import xgboost as xgb
    HAS_XGB = True
except Exception:
    HAS_XGB = False

t0 = time.time()
id_col = 'request_id'
target_col = 'requester_received_pizza'
ts_keys = ['unix_timestamp_of_request', 'unix_timestamp_of_request_utc']

train = pd.read_json('train.json')
test = pd.read_json('test.json')
y = train[target_col].astype(int).values
n = len(train)

# unix timestamp (with fallback)
if ts_keys[0] in train.columns:
    ts = train[ts_keys[0]].values
else:
    ts = train[ts_keys[1]].values

# Time order and recent slices
order = np.argsort(ts)  # ascending
cut35 = int(np.floor(0.65 * n))  # last 35%
cut45 = int(np.floor(0.55 * n))  # last 45%
idx_r35 = order[cut35:]
idx_r45 = order[cut45:]
print(f'MPNET recent -> n={n} | r35 size={len(idx_r35)} | r45 size={len(idx_r45)}')

# Cached features
meta_tr = np.load('meta_v1_tr.npy')  # (n, m_meta)
meta_te = np.load('meta_v1_te.npy')
emb_mp_tr = np.load('emb_mpnet_tr.npy')  # (n, d)
emb_mp_te = np.load('emb_mpnet_te.npy')

def fit_predict_mpnet_recent(idx_recent, out_path):
    X_tr_slice = np.hstack([emb_mp_tr[idx_recent], meta_tr[idx_recent]])
    X_te = np.hstack([emb_mp_te, meta_te])
    if HAS_XGB:
        idx_recent_sorted = np.sort(idx_recent)
        m = len(idx_recent_sorted)
        split = int(np.floor(0.80 * m))
        tr_idx = idx_recent_sorted[:split]
        va_idx = idx_recent_sorted[split:]
        Xtr = np.hstack([emb_mp_tr[tr_idx], meta_tr[tr_idx]])
        Xva = np.hstack([emb_mp_tr[va_idx], meta_tr[va_idx]])
        ytr = y[tr_idx]
        yva = y[va_idx]
        dtr = xgb.DMatrix(Xtr, label=ytr)
        dva = xgb.DMatrix(Xva, label=yva)
        dte = xgb.DMatrix(X_te)
        params = {
            'objective': 'binary:logistic',
            'eval_metric': 'auc',
            'max_depth': 3,
            'eta': 0.05,
            'subsample': 0.9,
            'colsample_bytree': 0.9,
            'reg_lambda': 1.0,
            'reg_alpha': 0.0,
            'min_child_weight': 1.0,
            'tree_method': 'hist',
            'seed': 42
        }
        bst = xgb.train(params, dtr, num_boost_round=300, evals=[(dva, 'valid')], early_stopping_rounds=50, verbose_eval=False)
        p = bst.predict(dte, iteration_range=(0, bst.best_iteration + 1)).astype(np.float32)
    else:
        lr = LogisticRegression(penalty='l2', solver='lbfgs', C=1.0, max_iter=1000, random_state=42)
        lr.fit(X_tr_slice, y[idx_recent])
        p = lr.predict_proba(X_te)[:, 1].astype(np.float32)
    np.save(out_path, p)
    print(f'Wrote {out_path} | mean={float(p.mean()):.6f}')

fit_predict_mpnet_recent(idx_r35, 'test_xgb_emb_mpnet_time_recent35.npy')
fit_predict_mpnet_recent(idx_r45, 'test_xgb_emb_mpnet_time_recent45.npy')

print(f'Done generating MPNet recent test-only files in {time.time()-t0:.1f}s')

In [ ]:
# S41-mpnet-3way: Add MPNet recency to S41, build gamma/r_low/r_high and 3-way hedges, then bias to means 0.36 and 0.40
import os, json, numpy as np, pandas as pd, time
from scipy.special import expit

assert 'clf' in globals() and 'X_test' in globals() and 'all_cols' in globals(), 'Run S41-rev (Cell 6) first.'

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs)
    lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi)
        m = expit(z+mid).mean()
        if abs(m - target_mean) < tol:
            return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, bias):
    return expit(logit_clip(probs) + bias)

with open('test.json', 'r') as f:
    te_json = json.load(f)
te_ids = [x.get('request_id') for x in te_json]

# Recent files map including MPNet
recent_map = {
    'LR_nosub': [
        'test_lr_time_nosub_meta_recent35.npy',
        'test_lr_time_nosub_meta_recent45.npy',
    ],
    'MiniLM': [
        'test_xgb_emb_meta_time_recent35.npy',
        'test_xgb_emb_meta_time_recent45.npy',
    ],
    'MPNet': [
        'test_xgb_emb_mpnet_time_recent35.npy',
        'test_xgb_emb_mpnet_time_recent45.npy',
    ],
}

col_to_idx = {c: i for i, c in enumerate(all_cols)}

def load_recent_avg_probs(files):
    arrs = []
    for fp in files:
        if os.path.exists(fp):
            a = np.load(fp)
            if a.ndim > 1: a = a.ravel()
            arrs.append(a)
    if not arrs:
        return None
    return np.mean(arrs, axis=0)

def apply_recency_sym(X_base, a_lr=0.0, a_mn=0.0, a_mp=0.0):
    Xr = X_base.copy()
    # For each base, interpolate in logit space between full X_test column and recent avg
    for name, alpha in [('LR_nosub', a_lr), ('MiniLM', a_mn), ('MPNet', a_mp)]:
        if alpha <= 0: continue
        if name not in col_to_idx: continue
        rec_files = recent_map.get(name, [])
        p_recent = load_recent_avg_probs(rec_files)
        if p_recent is None: continue
        idx = col_to_idx[name]
        z_full = X_base[:, idx]
        z_recent = logit_clip(p_recent)
        Xr[:, idx] = (1.0 - alpha)*z_full + alpha*z_recent
    return Xr

def save_sub(path, probs):
    df = pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': probs.astype(np.float32)})
    df.to_csv(path, index=False)
    print(f'Wrote {os.path.basename(path)} | mean={float(df.requester_received_pizza.mean()):.6f}')

t0 = time.time()
# 1) Build three variants: gamma (0,0,0), r_low (0.10 each), r_high (0.20 each)
Xg = X_test
pr_g = clf.predict_proba(Xg)[:, 1]
save_sub('submission_s41_final_gamma.csv', pr_g)

X_low = apply_recency_sym(X_test, a_lr=0.10, a_mn=0.10, a_mp=0.10)
pr_low = clf.predict_proba(X_low)[:, 1]
save_sub('submission_s41_final_r_low.csv', pr_low)

X_high = apply_recency_sym(X_test, a_lr=0.20, a_mn=0.20, a_mp=0.20)
pr_high = clf.predict_proba(X_high)[:, 1]
save_sub('submission_s41_final_r_high.csv', pr_high)

# Guardrail: recency should shift mean vs gamma
mg, ml, mh = float(pr_g.mean()), float(pr_low.mean()), float(pr_high.mean())
print('Means -> gamma:', mg, '| r_low:', ml, '| r_high:', mh)
if abs(ml - mg) <= 0.0005 or abs(mh - mg) <= 0.0005:
    print('WARNING: Recency shift is tiny; check recent files presence.')

# 2) 3-way logit hedge: gamma, r_low, r_high
zg = logit_clip(pr_g); zl = logit_clip(pr_low); zh = logit_clip(pr_high)
z3 = (zg + zl + zh) / 3.0
p3 = expit(z3)
save_sub('submission_s41_final_hedge3.csv', p3)

# 3) Bias shift final 3-way hedge to target means 0.36 and 0.40, promote 0.36 first
for tm in [0.36, 0.40]:
    b = find_bias_for_target_mean(p3, tm)
    p3m = apply_bias(p3, b)
    outp = f'submission_s41_final_hedge3_m{int(tm*100):03d}.csv'
    save_sub(outp, p3m)
    if tm == 0.36:
        pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': p3m.astype(np.float32)}).to_csv('submission.csv', index=False)
        print('PROMOTED: submission.csv <-', outp)

print(f'S41-mpnet-3way done in {time.time()-t0:.2f}s')

In [ ]:
# S41x: Blend S41 3-way hedge with S39 hedge2 (50/50 logit), then bias to mean 0.40 and promote
import os, numpy as np, pandas as pd
from scipy.special import expit

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs)
    lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi)
        m = expit(z+mid).mean()
        if abs(m - target_mean) < tol:
            return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, bias):
    return expit(logit_clip(probs) + bias)

def load_sub_norm(path):
    df = pd.read_csv(path)
    cols = [c.lower() for c in df.columns]
    if 'request_id' in cols:
        id_col = df.columns[cols.index('request_id')]
    elif 'id' in cols:
        id_col = df.columns[cols.index('id')]
    else:
        raise ValueError('No id/request_id in ' + path)
    prob_col = [c for c in df.columns if c != id_col][0]
    return df[[id_col, prob_col]].rename(columns={id_col:'request_id', prob_col:'requester_received_pizza'})

s41_path = 'submission_s41_final_hedge3.csv'
s39_path = 'submission_s39_hedge2_gamma_r24.csv'
assert os.path.exists(s41_path), f'Missing {s41_path}'
assert os.path.exists(s39_path), f'Missing {s39_path}'
s41 = load_sub_norm(s41_path)
s39 = load_sub_norm(s39_path)
assert np.all(s41.request_id.values == s39.request_id.values), 'ID mismatch between S41 and S39 hedges'

# 50/50 logit-average
z41 = logit_clip(s41.requester_received_pizza.values)
z39 = logit_clip(s39.requester_received_pizza.values)
z_blend = 0.5*(z41 + z39)
p_blend = expit(z_blend).astype(np.float32)
df_blend = pd.DataFrame({'request_id': s41.request_id.values, 'requester_received_pizza': p_blend})
df_blend.to_csv('submission_s41x_blend_s41_s39.csv', index=False)
print('Wrote submission_s41x_blend_s41_s39.csv | mean=', float(p_blend.mean()))

# Bias to mean 0.40
b = find_bias_for_target_mean(p_blend, 0.40)
p_bias = apply_bias(p_blend, b).astype(np.float32)
df_bias = pd.DataFrame({'request_id': s41.request_id.values, 'requester_received_pizza': p_bias})
df_bias.to_csv('submission_s41x_blend_s41_s39_m040.csv', index=False)
print('Wrote submission_s41x_blend_s41_s39_m040.csv | mean=', float(p_bias.mean()), '| bias=', float(b))

# Promote
df_bias.to_csv('submission.csv', index=False)
print('PROMOTED: submission.csv <- submission_s41x_blend_s41_s39_m040.csv')

In [ ]:
# S41-recent3: Recent-only 3-base ensemble (LR_nosub, MiniLM, MPNet), 50/50 with S41 3-way hedge, bias to 0.36/0.40
import os, numpy as np, pandas as pd
from scipy.special import expit

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def load_recent_avg(paths):
    arrs = []
    for fp in paths:
        if os.path.exists(fp):
            a = np.load(fp)
            if a.ndim > 1: a = a.ravel()
            arrs.append(a.astype(np.float64))
    return (np.mean(arrs, axis=0) if arrs else None)

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs)
    lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi)
        m = expit(z+mid).mean()
        if abs(m - target_mean) < tol:
            return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, bias):
    return expit(logit_clip(probs) + bias)

# Load recent35/45 for each base
p_lr = load_recent_avg(['test_lr_time_nosub_meta_recent35.npy','test_lr_time_nosub_meta_recent45.npy'])
p_mn = load_recent_avg(['test_xgb_emb_meta_time_recent35.npy','test_xgb_emb_meta_time_recent45.npy'])
p_mp = load_recent_avg(['test_xgb_emb_mpnet_time_recent35.npy','test_xgb_emb_mpnet_time_recent45.npy'])
assert p_lr is not None and p_mn is not None and p_mp is not None, 'Missing recent files for one of the bases'

# 3-base recent-only logit average
z_lr, z_mn, z_mp = logit_clip(p_lr), logit_clip(p_mn), logit_clip(p_mp)
z_recent3 = (z_lr + z_mn + z_mp) / 3.0
p_recent3 = expit(z_recent3).astype(np.float32)
pd.DataFrame({'request_id': pd.read_json('test.json')['request_id'].values, 'requester_received_pizza': p_recent3}).to_csv('submission_recent3.csv', index=False)
print('Wrote submission_recent3.csv | mean=', float(p_recent3.mean()))

# Load best S41 3-way hedge (unbiased) for blending
s41_3 = pd.read_csv('submission_s41_final_hedge3.csv') if os.path.exists('submission_s41_final_hedge3.csv') else pd.read_csv('submission_s41_meta_hedge3.csv')
p_s41 = s41_3['requester_received_pizza'].values.astype(np.float64)
assert len(p_s41) == len(p_recent3), 'Length mismatch S41 vs recent3'

# 50/50 logit blend of recent3 and S41 3-way
z_s41 = logit_clip(p_s41)
z_blend = 0.5*z_recent3 + 0.5*z_s41
p_blend = expit(z_blend).astype(np.float32)
df_blend = pd.DataFrame({'request_id': s41_3['request_id'].values, 'requester_received_pizza': p_blend})
df_blend.to_csv('submission_recent3_s41_blend.csv', index=False)
print('Wrote submission_recent3_s41_blend.csv | mean=', float(p_blend.mean()))

# Bias-shift to target means 0.36 and 0.40; promote 0.36
for tm in [0.36, 0.40]:
    b = find_bias_for_target_mean(p_blend, tm)
    p_b = apply_bias(p_blend, b).astype(np.float32)
    outp = f'submission_recent3_s41_blend_m{int(tm*100):03d}.csv'
    pd.DataFrame({'request_id': s41_3['request_id'].values, 'requester_received_pizza': p_b}).to_csv(outp, index=False)
    print(f'Wrote {outp} | mean={p_b.mean():.6f} | bias={b:.4f}')
    if tm == 0.36:
        pd.DataFrame({'request_id': s41_3['request_id'].values, 'requester_received_pizza': p_b}).to_csv('submission.csv', index=False)
        print('PROMOTED: submission.csv <-', outp)

In [ ]:
# S41-submit-034: Bias S41 3-way hedge to mean 0.34 and promote
import os, numpy as np, pandas as pd
from scipy.special import expit

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, it=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(it):
        mid = 0.5*(lo+hi); m = expit(z+mid).mean()
        if abs(m-target_mean) < tol: return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, b):
    return expit(logit_clip(probs)+b)

src = 'submission_s41_final_hedge3.csv' if os.path.exists('submission_s41_final_hedge3.csv') else 'submission_s41_meta_hedge3.csv'
s = pd.read_csv(src)
b = find_bias_for_target_mean(s.requester_received_pizza.values, 0.34)
s['requester_received_pizza'] = apply_bias(s.requester_received_pizza.values, b).astype(np.float32)
s.to_csv('submission.csv', index=False)
print('Promoted submission.csv | mean=', float(s.requester_received_pizza.mean()), '| bias=', float(b))

In [ ]:
# S41-diagnostic: Last-block AUC per base vs gamma-weighted overall; flag degraders
import numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

required = ['X_oof','all_cols','block_id','train_idx','y_tr','W_tr','oof_pred']
missing = [k for k in required if k not in globals()]
assert not missing, f'Missing artifacts from S41-rev: {missing}. Re-run Cell 6.'

# Define last validated block mask on training indices
mask_last = (block_id[train_idx] == 5)
y_last = y_tr[mask_last]
X_oof_tr = X_oof[train_idx]

rows = []
for i, name in enumerate(all_cols):
    if name in ['log1p_text_len','account_age_days']:
        continue
    z = X_oof_tr[:, i]
    try:
        auc_overall = roc_auc_score(y_tr, z, sample_weight=W_tr)
        auc_last = roc_auc_score(y_last, z[mask_last])
        rows.append((name, float(auc_overall), float(auc_last), float(auc_overall - auc_last)))
    except Exception as e:
        rows.append((name, np.nan, np.nan, np.nan))

# Add stacker itself
try:
    auc_overall = roc_auc_score(y_tr, oof_pred, sample_weight=W_tr)
    auc_last = roc_auc_score(y_last, oof_pred[mask_last])
    rows.append(('S41_Stacker', float(auc_overall), float(auc_last), float(auc_overall - auc_last)))
except Exception:
    pass

df = pd.DataFrame(rows, columns=['Model','AUC_overall','AUC_last','Drop']).sort_values(['Drop','AUC_last'], ascending=[False, True])
print(df.to_string(index=False))
df.to_csv('s41_last_block_diagnostic.csv', index=False)
print('Saved s41_last_block_diagnostic.csv')

# Heuristic flags: Drop > 0.03 or AUC_last < 0.60
flags = df[(df['Model']!='S41_Stacker') & ((df['Drop']>0.03) | (df['AUC_last']<0.60))]
print('\nFlagged degraders (Drop>0.03 or AUC_last<0.60):')
print(flags.to_string(index=False))

In [ ]:
# S41-mpnet-stronger: Add stronger symmetric recency (0.15, 0.25, 0.30), build 4-way hedge incl. gamma, bias to 0.34 and promote
import os, json, numpy as np, pandas as pd, time
from scipy.special import expit

assert 'clf' in globals() and 'X_test' in globals() and 'all_cols' in globals(), 'Run S41-rev (Cell 6) first.'

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs)
    lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi)
        m = expit(z+mid).mean()
        if abs(m - target_mean) < tol:
            return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, bias):
    return expit(logit_clip(probs) + bias)

with open('test.json', 'r') as f:
    te_json = json.load(f)
te_ids = [x.get('request_id') for x in te_json]

# Recent files map including MPNet
recent_map = {
    'LR_nosub': [
        'test_lr_time_nosub_meta_recent35.npy',
        'test_lr_time_nosub_meta_recent45.npy',
    ],
    'MiniLM': [
        'test_xgb_emb_meta_time_recent35.npy',
        'test_xgb_emb_meta_time_recent45.npy',
    ],
    'MPNet': [
        'test_xgb_emb_mpnet_time_recent35.npy',
        'test_xgb_emb_mpnet_time_recent45.npy',
    ],
}

col_to_idx = {c: i for i, c in enumerate(all_cols)}

def load_recent_avg_probs(files):
    arrs = []
    for fp in files:
        if os.path.exists(fp):
            a = np.load(fp)
            if a.ndim > 1: a = a.ravel()
            arrs.append(a)
    if not arrs:
        return None
    return np.mean(arrs, axis=0)

def apply_recency_sym(X_base, a_lr=0.0, a_mn=0.0, a_mp=0.0):
    Xr = X_base.copy()
    for name, alpha in [('LR_nosub', a_lr), ('MiniLM', a_mn), ('MPNet', a_mp)]:
        if alpha <= 0: continue
        if name not in col_to_idx: continue
        p_recent = load_recent_avg_probs(recent_map.get(name, []))
        if p_recent is None: continue
        idx = col_to_idx[name]
        z_full = X_base[:, idx]
        z_recent = logit_clip(p_recent)
        Xr[:, idx] = (1.0 - alpha)*z_full + alpha*z_recent
    return Xr

def save_sub(path, probs):
    df = pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': probs.astype(np.float32)})
    df.to_csv(path, index=False)
    print(f'Wrote {os.path.basename(path)} | mean={float(df.requester_received_pizza.mean()):.6f}')

t0 = time.time()
# gamma (no recency)
pr_g = clf.predict_proba(X_test)[:, 1]
save_sub('submission_s41_final_gamma.csv', pr_g)

# Build symmetric recency variants
variants = {
    'r010': (0.10, 0.10, 0.10),
    'r015': (0.15, 0.15, 0.15),
    'r020': (0.20, 0.20, 0.20),
    'r030': (0.30, 0.30, 0.30),
}
preds = {'gamma': pr_g}
for tag, (alr, amn, amp) in variants.items():
    Xv = apply_recency_sym(X_test, a_lr=alr, a_mn=amn, a_mp=amp)
    pv = clf.predict_proba(Xv)[:, 1]
    preds[tag] = pv
    save_sub(f'submission_s41_final_{tag}.csv', pv)

# Hedge: 4-way logit hedge across gamma + r010 + r020 + r030
zg = logit_clip(preds['gamma'])
z010 = logit_clip(preds['r010'])
z020 = logit_clip(preds['r020'])
z030 = logit_clip(preds['r030'])
z4 = (zg + z010 + z020 + z030) / 4.0
p4 = expit(z4)
save_sub('submission_s41_final_hedge4.csv', p4)

# Bias to mean 0.34 and promote
b = find_bias_for_target_mean(p4, 0.34)
p4m = apply_bias(p4, b)
save_sub('submission_s41_final_hedge4_m034.csv', p4m)
pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': p4m.astype(np.float32)}).to_csv('submission.csv', index=False)
print('PROMOTED: submission.csv <- submission_s41_final_hedge4_m034.csv | took', f'{time.time()-t0:.2f}s')

In [ ]:
# Promote unshifted S41 3-way hedge to submission.csv
import pandas as pd
src = 'submission_s41_final_hedge3.csv'
df = pd.read_csv(src)
df.to_csv('submission.csv', index=False)
print('Promoted', src, 'to submission.csv | mean=', float(df['requester_received_pizza'].mean()))

In [ ]:
# Promote r_high variant (single model) biased to mean 0.34
import numpy as np, pandas as pd, os
from scipy.special import expit

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, it=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(it):
        mid = 0.5*(lo+hi); m = expit(z+mid).mean()
        if abs(m-target_mean) < tol: return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, b):
    return expit(logit_clip(probs)+b)

src = 'submission_s41_final_r_high.csv'
assert os.path.exists(src), f'Missing {src}; run S41-mpnet-3way (Cell 19/24) first.'
s = pd.read_csv(src)
b = find_bias_for_target_mean(s.requester_received_pizza.values, 0.34)
s['requester_received_pizza'] = apply_bias(s.requester_received_pizza.values, b).astype(np.float32)
s.to_csv('submission_s41_final_r_high_m034.csv', index=False)
s.to_csv('submission.csv', index=False)
print('Promoted submission_s41_final_r_high_m034.csv to submission.csv | mean=', float(s.requester_received_pizza.mean()), '| bias=', float(b))

In [ ]:
# S41d-bias-032: Bias-shift submission_s41_meta_hedge2.csv to mean 0.32 and promote
import numpy as np, pandas as pd, os
from scipy.special import expit

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, it=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(it):
        mid = 0.5*(lo+hi); m = expit(z+mid).mean()
        if abs(m - target_mean) < tol: return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, b):
    return expit(logit_clip(probs)+b)

src = 'submission_s41_meta_hedge2.csv'
assert os.path.exists(src), f'Missing {src}; run S41-rev (Cell 6) first.'
df = pd.read_csv(src)
# Normalize columns
cols = [c.lower() for c in df.columns]
if 'request_id' in cols:
    id_col = df.columns[cols.index('request_id')]
elif 'id' in cols:
    id_col = df.columns[cols.index('id')]
else:
    raise ValueError('No id/request_id column in source submission')
prob_col = [c for c in df.columns if c != id_col][0]
probs = df[prob_col].values.astype(float)

target_mean = 0.32
b = find_bias_for_target_mean(probs, target_mean)
probs_b = apply_bias(probs, b).astype(np.float32)
out = pd.DataFrame({'request_id': df[id_col].values, 'requester_received_pizza': probs_b})
out.to_csv('submission_s41_meta_hedge2_m032.csv', index=False)
out.to_csv('submission.csv', index=False)
print(f'Promoted submission_s41_meta_hedge2_m032.csv -> submission.csv | mean={float(out.requester_received_pizza.mean()):.6f} | bias={float(b):.4f}')

In [ ]:
# S41-final-asym: Asymmetric recency (LR_nosub small, MiniLM/MPNet strong), 3-way hedge, bias to 0.30/0.32
import os, json, numpy as np, pandas as pd, time
from scipy.special import expit

assert 'clf' in globals() and 'X_test' in globals() and 'all_cols' in globals(), 'Run S41-rev (Cell 6) first.'

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs)
    lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi)
        m = expit(z+mid).mean()
        if abs(m - target_mean) < tol:
            return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, bias):
    return expit(logit_clip(probs) + bias)

with open('test.json', 'r') as f:
    te_json = json.load(f)
te_ids = [x.get('request_id') for x in te_json]

# Recent files map including MPNet
recent_map = {
    'LR_nosub': [
        'test_lr_time_nosub_meta_recent35.npy',
        'test_lr_time_nosub_meta_recent45.npy',
    ],
    'MiniLM': [
        'test_xgb_emb_meta_time_recent35.npy',
        'test_xgb_emb_meta_time_recent45.npy',
    ],
    'MPNet': [
        'test_xgb_emb_mpnet_time_recent35.npy',
        'test_xgb_emb_mpnet_time_recent45.npy',
    ],
}

col_to_idx = {c: i for i, c in enumerate(all_cols)}

def load_recent_avg_probs(files):
    arrs = []
    for fp in files:
        if os.path.exists(fp):
            a = np.load(fp)
            if a.ndim > 1: a = a.ravel()
            arrs.append(a)
    if not arrs:
        return None
    return np.mean(arrs, axis=0)

def apply_recency_asym(X_base, a_lr=0.0, a_mn=0.0, a_mp=0.0):
    Xr = X_base.copy()
    for name, alpha in [('LR_nosub', a_lr), ('MiniLM', a_mn), ('MPNet', a_mp)]:
        if alpha <= 0: continue
        if name not in col_to_idx: continue
        p_recent = load_recent_avg_probs(recent_map.get(name, []))
        if p_recent is None: continue
        idx = col_to_idx[name]
        z_full = X_base[:, idx]
        z_recent = logit_clip(p_recent)
        Xr[:, idx] = (1.0 - alpha)*z_full + alpha*z_recent
    return Xr

def save_sub(path, probs):
    df = pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': probs.astype(np.float32)})
    df.to_csv(path, index=False)
    print(f'Wrote {os.path.basename(path)} | mean={float(df.requester_received_pizza.mean()):.6f}')

t0 = time.time()
# 1) Build three variants with asymmetric alphas per expert advice:
# gamma: (0, 0, 0) | r_low: (0.00, 0.15, 0.20) | r_high: (0.05, 0.25, 0.30)
pr_g = clf.predict_proba(X_test)[:, 1]
save_sub('submission_s41_final_gamma.csv', pr_g)

X_low = apply_recency_asym(X_test, a_lr=0.00, a_mn=0.15, a_mp=0.20)
pr_low = clf.predict_proba(X_low)[:, 1]
save_sub('submission_s41_final_asym_r_low.csv', pr_low)

X_high = apply_recency_asym(X_test, a_lr=0.05, a_mn=0.25, a_mp=0.30)
pr_high = clf.predict_proba(X_high)[:, 1]
save_sub('submission_s41_final_asym_r_high.csv', pr_high)

mg, ml, mh = float(pr_g.mean()), float(pr_low.mean()), float(pr_high.mean())
print('Means -> gamma:', mg, '| r_low:', ml, '| r_high:', mh)

# 2) 3-way logit hedge: gamma, r_low, r_high
zg, zl, zh = logit_clip(pr_g), logit_clip(pr_low), logit_clip(pr_high)
z3 = (zg + zl + zh) / 3.0
p3 = expit(z3)
save_sub('submission_s41_final_hedge3_asym.csv', p3)

# 3) Bias to 0.30 and 0.32; promote 0.30
for tm in [0.30, 0.32]:
    b = find_bias_for_target_mean(p3, tm)
    p3m = apply_bias(p3, b)
    outp = f'submission_s41_final_hedge3_asym_m{int(tm*100):03d}.csv'
    save_sub(outp, p3m)
    if abs(tm - 0.30) < 1e-9:
        pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': p3m.astype(np.float32)}).to_csv('submission.csv', index=False)
        print('PROMOTED: submission.csv <-', outp)

print(f'S41-final-asym done in {time.time()-t0:.2f}s')

In [ ]:
# S42-xgb: XGBoost stacker on pruned bases + asymmetric recency variants + 3-way hedge + bias to 0.30
import os, json, time
import numpy as np
import pandas as pd
from scipy.special import expit

try:
    import xgboost as xgb
except Exception as e:
    raise RuntimeError('XGBoost not installed; install xgboost to run S42-xgb')

t0 = time.time()
# Guards: require artifacts from S41 (features built in memory)
required = ['X_oof','X_test','all_cols','mask_full','y','W','tr_json','te_json']
missing = [k for k in required if k not in globals()]
assert not missing, f'Missing artifacts from S41-rev: {missing}. Run Cell 6 first.'

print('S42-xgb: starting XGBoost stacker on features:', all_cols)

# Train on validated mask (blocks 1..5) with gamma weights
train_idx = np.where(mask_full)[0]
X_tr = X_oof[train_idx].astype(np.float32)
y_tr = np.array([int(x.get('requester_received_pizza') or 0) for x in tr_json], dtype=int)[train_idx]
W_tr = W[train_idx].astype(np.float32)

dtr = xgb.DMatrix(X_tr, label=y_tr, weight=W_tr, feature_names=[str(c) for c in all_cols])
dte_full = xgb.DMatrix(X_test.astype(np.float32), feature_names=[str(c) for c in all_cols])

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'eta': 0.03,
    'max_depth': 6,
    'min_child_weight': 100,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'reg_alpha': 1.0,
    'reg_lambda': 10.0,
    'tree_method': 'hist',
    'seed': 42
}
bst = xgb.train(params, dtr, num_boost_round=1200, verbose_eval=False)
print('XGB trained. Building variants...')

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi); m = expit(z+mid).mean()
        if abs(m - target_mean) < tol: return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, b):
    return expit(logit_clip(probs) + b)

# Recent files map
recent_map = {
    'LR_nosub': [
        'test_lr_time_nosub_meta_recent35.npy',
        'test_lr_time_nosub_meta_recent45.npy',
    ],
    'MiniLM': [
        'test_xgb_emb_meta_time_recent35.npy',
        'test_xgb_emb_meta_time_recent45.npy',
    ],
    'MPNet': [
        'test_xgb_emb_mpnet_time_recent35.npy',
        'test_xgb_emb_mpnet_time_recent45.npy',
    ],
}
col_to_idx = {c: i for i, c in enumerate(all_cols)}

def load_recent_avg_probs(files):
    arrs = []
    for fp in files:
        if os.path.exists(fp):
            a = np.load(fp)
            if a.ndim > 1: a = a.ravel()
            arrs.append(a)
    if not arrs: return None
    return np.mean(arrs, axis=0)

def apply_recency_asym_to_Xtest(X_base, a_lr=0.0, a_mn=0.0, a_mp=0.0):
    Xr = X_base.copy()
    for name, alpha in [('LR_nosub', a_lr), ('MiniLM', a_mn), ('MPNet', a_mp)]:
        if alpha <= 0: continue
        if name not in col_to_idx: continue
        p_recent = load_recent_avg_probs(recent_map.get(name, []))
        if p_recent is None: continue
        idx = col_to_idx[name]
        z_full = X_base[:, idx]
        z_recent = logit_clip(p_recent)
        Xr[:, idx] = (1.0 - alpha)*z_full + alpha*z_recent
    return Xr

te_ids = [x.get('request_id') for x in te_json]
def save_sub(path, probs):
    df = pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': probs.astype(np.float32)})
    df.to_csv(path, index=False)
    print(f'Wrote {os.path.basename(path)} | mean={float(df.requester_received_pizza.mean()):.6f}')

# 1) gamma (no recency)
p_gamma = bst.predict(dte_full).astype(np.float32)
save_sub('submission_s42x_meta_gamma.csv', p_gamma)

# 2) Asymmetric recency variants: r_low (0.00,0.15,0.20), r_high (0.05,0.25,0.30)
X_low = apply_recency_asym_to_Xtest(X_test, a_lr=0.00, a_mn=0.15, a_mp=0.20)
p_low = bst.predict(xgb.DMatrix(X_low.astype(np.float32), feature_names=[str(c) for c in all_cols])).astype(np.float32)
save_sub('submission_s42x_meta_asym_r_low.csv', p_low)

X_high = apply_recency_asym_to_Xtest(X_test, a_lr=0.05, a_mn=0.25, a_mp=0.30)
p_high = bst.predict(xgb.DMatrix(X_high.astype(np.float32), feature_names=[str(c) for c in all_cols])).astype(np.float32)
save_sub('submission_s42x_meta_asym_r_high.csv', p_high)

print('Means -> gamma:', float(p_gamma.mean()), '| r_low:', float(p_low.mean()), '| r_high:', float(p_high.mean()))

# 3) 3-way logit hedge and bias to 0.30 (promote) and 0.32 (portfolio file)
zg, zl, zh = logit_clip(p_gamma), logit_clip(p_low), logit_clip(p_high)
p3 = expit((zg + zl + zh)/3.0).astype(np.float32)
save_sub('submission_s42x_meta_hedge3.csv', p3)

for tm in [0.30, 0.32]:
    b = find_bias_for_target_mean(p3, tm)
    p3m = apply_bias(p3, b).astype(np.float32)
    outp = f'submission_s42x_meta_hedge3_m{int(tm*100):03d}.csv'
    save_sub(outp, p3m)
    if abs(tm - 0.30) < 1e-9:
        pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': p3m}).to_csv('submission.csv', index=False)
        print('PROMOTED: submission.csv <-', outp)

print(f'S42-xgb done in {time.time()-t0:.2f}s')

In [ ]:
# S41-submit-028: Bias asymmetric 3-way hedge to mean 0.28 and promote
import os, numpy as np, pandas as pd
from scipy.special import expit

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, it=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(it):
        mid = 0.5*(lo+hi); m = expit(z+mid).mean()
        if abs(m - target_mean) < tol: return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, b):
    return expit(logit_clip(probs)+b)

src = 'submission_s41_final_hedge3_asym.csv'
if not os.path.exists(src):
    # fallback to symmetric hedge if asym not present
    src = 'submission_s41_final_hedge3.csv' if os.path.exists('submission_s41_final_hedge3.csv') else 'submission_s42x_meta_hedge3.csv'
s = pd.read_csv(src)
b = find_bias_for_target_mean(s['requester_received_pizza'].values, 0.28)
s['requester_received_pizza'] = apply_bias(s['requester_received_pizza'].values, b).astype(np.float32)
s.to_csv('submission_s41_final_hedge3_asym_m028.csv', index=False)
s.to_csv('submission.csv', index=False)
print('Promoted', src, '-> submission_s41_final_hedge3_asym_m028.csv -> submission.csv | mean=', float(s['requester_received_pizza'].mean()), '| bias=', float(b))

In [ ]:
# S43-rank5: Diversified rank-average of top-5 bases -> bias to mean 0.35 and promote
import os, json, numpy as np, pandas as pd
from scipy.special import expit

print('S43-rank5: building rank-average ensemble (LR_nosub, Dense_v1, Meta, MiniLM, MPNet) ...')
with open('test.json', 'r') as f:
    te_json = json.load(f)
te_ids = [x.get('request_id') for x in te_json]

def load_probs(fp):
    a = np.load(fp)
    if a.ndim > 1: a = a.ravel()
    return a.astype(np.float64)

bases = [
    ('LR_nosub', 'test_lr_time_nosub_meta.npy'),
    ('Dense_v1', 'test_xgb_dense_time.npy'),
    ('Meta', 'test_xgb_meta_time.npy'),
    ('MiniLM', 'test_xgb_emb_meta_time.npy'),
    ('MPNet', 'test_xgb_emb_mpnet_time.npy'),
]

arrs = []
for name, fp in bases:
    if not os.path.exists(fp):
        raise FileNotFoundError(f'Missing {fp} for base {name}')
    p = load_probs(fp)
    arrs.append(p)
    print(f'Loaded {name}: mean={float(p.mean()):.6f}')

arrs = np.stack(arrs, axis=1)  # (m, 5)
m = arrs.shape[0]

def rank01(x):
    # deterministic rank to [0,1] without scipy; ties handled by average of positions of equal values
    # For continuous preds ties are rare; fallback to simple order-based ranks
    order = np.argsort(x, kind='mergesort')
    ranks = np.empty_like(order, dtype=np.float64)
    ranks[order] = np.arange(m, dtype=np.float64)
    return ranks / (m - 1) if m > 1 else np.zeros_like(x, dtype=np.float64)

rank_cols = np.column_stack([rank01(arrs[:, j]) for j in range(arrs.shape[1])])  # (m,5)
rank_avg = rank_cols.mean(axis=1).astype(np.float64)  # in [0,1]

df_rank = pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': rank_avg.astype(np.float32)})
df_rank.to_csv('submission_rank5.csv', index=False)
print('Wrote submission_rank5.csv | mean=', float(df_rank.requester_received_pizza.mean()))

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=np.float64)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))
def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi); m = expit(z+mid).mean()
        if abs(m - target_mean) < tol: return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)
def apply_bias(probs, b):
    return expit(logit_clip(probs) + b)

target_mean = 0.35
b = find_bias_for_target_mean(rank_avg, target_mean)
rank_avg_b = apply_bias(rank_avg, b).astype(np.float32)
df_b = pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': rank_avg_b})
df_b.to_csv('submission_rank5_m035.csv', index=False)
df_b.to_csv('submission.csv', index=False)
print(f'PROMOTED: submission.csv <- submission_rank5_m035.csv | mean={float(df_b.requester_received_pizza.mean()):.6f} | bias={float(b):.4f}')

In [ ]:
# S44-hybrid: 50/50 logit hedge of S41 asym 3-way and rank-average; bias to 0.30 and 0.32
import os, numpy as np, pandas as pd
from scipy.special import expit

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def load_sub_norm(path):
    df = pd.read_csv(path)
    cols = [c.lower() for c in df.columns]
    if 'request_id' in cols:
        id_col = df.columns[cols.index('request_id')]
    elif 'id' in cols:
        id_col = df.columns[cols.index('id')]
    else:
        raise ValueError('No id/request_id in ' + path)
    prob_col = [c for c in df.columns if c != id_col][0]
    out = df[[id_col, prob_col]].rename(columns={id_col:'request_id', prob_col:'requester_received_pizza'})
    return out

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi); m = expit(z+mid).mean()
        if abs(m - target_mean) < tol: return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, b):
    return expit(logit_clip(probs) + b)

# Inputs: S41 asym 3-way hedge (unbiased) and rank-average (unbiased)
path_stack = 'submission_s41_final_hedge3_asym.csv'
path_rank = 'submission_rank5.csv'
assert os.path.exists(path_stack), f'Missing {path_stack}; run Cell 28 first.'
assert os.path.exists(path_rank), f'Missing {path_rank}; run Cell 31 first.'

s_stack = load_sub_norm(path_stack)
s_rank = load_sub_norm(path_rank)
assert np.all(s_stack.request_id.values == s_rank.request_id.values), 'ID mismatch between stack and rank submissions'

# 50/50 logit hedge
z1 = logit_clip(s_stack.requester_received_pizza.values)
z2 = logit_clip(s_rank.requester_received_pizza.values)
z = 0.5*(z1 + z2)
p = expit(z).astype(np.float32)
df_h = pd.DataFrame({'request_id': s_stack.request_id.values, 'requester_received_pizza': p})
df_h.to_csv('submission_s44_hybrid_50_50.csv', index=False)
print('Wrote submission_s44_hybrid_50_50.csv | mean=', float(p.mean()))

# Bias to 0.30 and 0.32; promote 0.30
for tm in [0.30, 0.32]:
    b = find_bias_for_target_mean(p, tm)
    pm = apply_bias(p, b).astype(np.float32)
    outp = f'submission_s44_hybrid_50_50_m{int(tm*100):03d}.csv'
    pd.DataFrame({'request_id': s_stack.request_id.values, 'requester_received_pizza': pm}).to_csv(outp, index=False)
    print(f'Wrote {outp} | mean={pm.mean():.6f} | bias={b:.4f}')
    if abs(tm - 0.30) < 1e-9:
        pd.DataFrame({'request_id': s_stack.request_id.values, 'requester_received_pizza': pm}).to_csv('submission.csv', index=False)
        print('PROMOTED: submission.csv <-', outp)

In [ ]:
# S45-eq5: Simple 5-base logit-average (LR_nosub, Dense_v1, Meta, MiniLM, MPNet) -> bias to 0.30 and 0.32
import os, numpy as np, pandas as pd
from scipy.special import expit

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi); m = expit(z+mid).mean()
        if abs(m - target_mean) < tol: return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, b):
    return expit(logit_clip(probs) + b)

bases = [
    ('LR_nosub', 'test_lr_time_nosub_meta.npy'),
    ('Dense_v1', 'test_xgb_dense_time.npy'),
    ('Meta', 'test_xgb_meta_time.npy'),
    ('MiniLM', 'test_xgb_emb_meta_time.npy'),
    ('MPNet', 'test_xgb_emb_mpnet_time.npy'),
]

arrs = []
for name, fp in bases:
    if not os.path.exists(fp):
        raise FileNotFoundError(f'Missing {fp} for base {name}')
    a = np.load(fp)
    if a.ndim > 1: a = a.ravel()
    arrs.append(a.astype(np.float64))

Z = np.column_stack([logit_clip(a) for a in arrs])  # (m,5)
z_mean = Z.mean(axis=1)
p_eq5 = expit(z_mean).astype(np.float32)

te_ids = pd.read_json('test.json')['request_id'].values
df = pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': p_eq5})
df.to_csv('submission_eq5_logitavg.csv', index=False)
print('Wrote submission_eq5_logitavg.csv | mean=', float(df.requester_received_pizza.mean()))

for tm in [0.30, 0.32]:
    b = find_bias_for_target_mean(p_eq5, tm)
    pm = apply_bias(p_eq5, b).astype(np.float32)
    outp = f'submission_eq5_logitavg_m{int(tm*100):03d}.csv'
    pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': pm}).to_csv(outp, index=False)
    print(f'Wrote {outp} | mean={pm.mean():.6f} | bias={b:.4f}')
    if abs(tm - 0.30) < 1e-9:
        pd.DataFrame({'request_id': te_ids, 'requester_received_pizza': pm}).to_csv('submission.csv', index=False)
        print('PROMOTED: submission.csv <-', outp)

In [ ]:
# Promote time-aware stacker 0.32 mean variant to submission.csv
import pandas as pd, os
src = 'submission_s41_time_meta_gamma_m032.csv'
assert os.path.exists(src), f'Missing {src}; run Cell 6 first.'
df = pd.read_csv(src)
df.to_csv('submission.csv', index=False)
print('Promoted', src, 'to submission.csv | mean=', float(df['requester_received_pizza'].mean()))

In [ ]:
# S43-rank5-033: Bias existing rank-average to mean 0.33 and promote
import os, numpy as np, pandas as pd
from scipy.special import expit

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi); m = expit(z+mid).mean()
        if abs(m - target_mean) < tol: return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, b):
    return expit(logit_clip(probs) + b)

src = 'submission_rank5.csv'
assert os.path.exists(src), 'Missing submission_rank5.csv; run Cell 31 first.'
s = pd.read_csv(src)
probs = s['requester_received_pizza'].values.astype(float)
b = find_bias_for_target_mean(probs, 0.33)
s['requester_received_pizza'] = apply_bias(probs, b).astype(np.float32)
s.to_csv('submission_rank5_m033.csv', index=False)
s.to_csv('submission.csv', index=False)
print('PROMOTED: submission.csv <- submission_rank5_m033.csv | mean=', float(s['requester_received_pizza'].mean()), '| bias=', float(b))

In [ ]:
# Promote time-aware stacker 0.28 mean variant to submission.csv
import pandas as pd, os
src = 'submission_s41_time_meta_gamma_m028.csv'
assert os.path.exists(src), f'Missing {src}; run Cell 6 first.'
df = pd.read_csv(src)
df.to_csv('submission.csv', index=False)
print('Promoted', src, 'to submission.csv | mean=', float(df['requester_received_pizza'].mean()))

In [ ]:
# S46-hybrid-time-rank5: 50/50 logit hedge of time-aware stacker (unbiased) and rank-average; bias to 0.30/0.32
import os, numpy as np, pandas as pd
from scipy.special import expit

def logit_clip(p, eps=1e-6):
    p = np.asarray(p, dtype=float)
    p = np.clip(p, eps, 1 - eps)
    return np.log(p/(1-p))

def load_sub_norm(path):
    df = pd.read_csv(path)
    cols = [c.lower() for c in df.columns]
    if 'request_id' in cols:
        id_col = df.columns[cols.index('request_id')]
    elif 'id' in cols:
        id_col = df.columns[cols.index('id')]
    else:
        raise ValueError('No id/request_id in ' + path)
    prob_col = [c for c in df.columns if c != id_col][0]
    out = df[[id_col, prob_col]].rename(columns={id_col:'request_id', prob_col:'requester_received_pizza'})
    return out

def find_bias_for_target_mean(probs, target_mean, tol=1e-6, max_iter=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(max_iter):
        mid = 0.5*(lo+hi); m = expit(z+mid).mean()
        if abs(m - target_mean) < tol: return mid
        if m < target_mean: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

def apply_bias(probs, b):
    return expit(logit_clip(probs) + b)

# Inputs: time-aware stacker (unbiased) and rank-average (unbiased)
path_time = 'submission_s41_time_meta_gamma.csv'
path_rank = 'submission_rank5.csv'
assert os.path.exists(path_time), f'Missing {path_time}; run Cell 6 first.'
assert os.path.exists(path_rank), f'Missing {path_rank}; run Cell 31 first.'

s_time = load_sub_norm(path_time)
s_rank = load_sub_norm(path_rank)
assert np.all(s_time.request_id.values == s_rank.request_id.values), 'ID mismatch between time-aware and rank submissions'

# 50/50 logit hedge
z1 = logit_clip(s_time.requester_received_pizza.values)
z2 = logit_clip(s_rank.requester_received_pizza.values)
z = 0.5*(z1 + z2)
p = expit(z).astype(np.float32)
df_h = pd.DataFrame({'request_id': s_time.request_id.values, 'requester_received_pizza': p})
df_h.to_csv('submission_s46_hybrid_time_rank.csv', index=False)
print('Wrote submission_s46_hybrid_time_rank.csv | mean=', float(p.mean()))

# Bias to 0.30 and 0.32; promote 0.30
for tm in [0.30, 0.32]:
    b = find_bias_for_target_mean(p, tm)
    pm = apply_bias(p, b).astype(np.float32)
    outp = f'submission_s46_hybrid_time_rank_m{int(tm*100):03d}.csv'
    pd.DataFrame({'request_id': s_time.request_id.values, 'requester_received_pizza': pm}).to_csv(outp, index=False)
    print(f'Wrote {outp} | mean={pm.mean():.6f} | bias={b:.4f}')
    if abs(tm - 0.30) < 1e-9:
        pd.DataFrame({'request_id': s_time.request_id.values, 'requester_received_pizza': pm}).to_csv('submission.csv', index=False)
        print('PROMOTED: submission.csv <-', outp)

In [ ]:
# S47-LR-OOF: True forward-chaining LogisticRegression stacker with time interactions + recency hedges
import os, time, math
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

t0 = time.time()
print('S47-LR-OOF: starting...')
id_col = 'request_id'; target_col = 'requester_received_pizza'
train = pd.read_json('train.json'); test = pd.read_json('test.json')
y = train[target_col].astype(int).values
ntr, nte = len(train), len(test)

def to_logit(p, eps=1e-6):
    p = np.clip(np.asarray(p, dtype=np.float64), eps, 1-eps)
    return np.log(p/(1-p))
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

# Time ordering and contiguous blocks
ts_col = 'unix_timestamp_of_request' if 'unix_timestamp_of_request' in train.columns else 'unix_timestamp_of_request_utc'
order = np.argsort(train[ts_col].values)
k = 6
blocks = np.array_split(order, k)

# Load base OOF/test probs and convert to logits
bases = [
    ('LR_nosub', 'oof_lr_time_nosub_meta.npy', 'test_lr_time_nosub_meta.npy'),
    ('Dense_v1', 'oof_xgb_dense_time.npy', 'test_xgb_dense_time.npy'),
    ('Meta',    'oof_xgb_meta_time.npy',  'test_xgb_meta_time.npy'),
    ('MiniLM',  'oof_xgb_emb_meta_time.npy', 'test_xgb_emb_meta_time.npy'),
    ('MPNet',   'oof_xgb_emb_mpnet_time.npy', 'test_xgb_emb_mpnet_time.npy'),
    ('E5_meta', 'oof_xgb_e5_meta_time.npy', 'test_xgb_e5_meta_time.npy'),
]
names, Z_oof_list, Z_te_list = [], [], []
for name, oof_fp, te_fp in bases:
    if (not os.path.exists(oof_fp)) or (not os.path.exists(te_fp)):
        print(f'Skipping {name}: missing files')
        continue
    oof = np.load(oof_fp); te = np.load(te_fp)
    if oof.ndim>1: oof=oof.ravel()
    if te.ndim>1: te=te.ravel()
    if len(oof) != ntr:
        print(f'Skipping {name}: OOF length mismatch {len(oof)} != {ntr}')
        continue
    names.append(name)
    Z_oof_list.append(to_logit(oof)); Z_te_list.append(to_logit(te))
print('Included bases:', names)
Xb_tr = np.column_stack(Z_oof_list).astype(np.float64)
Xb_te = np.column_stack(Z_te_list).astype(np.float64)
nb = Xb_tr.shape[1]

# Meta features
def get_text(df):
    title = df.get('request_title', pd.Series(['']*len(df))).fillna('').astype(str)
    body = df.get('request_text_edit_aware', df.get('request_text', pd.Series(['']*len(df)))).fillna('').astype(str)
    return title + '\n' + body
tr_txt = get_text(train); te_txt = get_text(test)
tr_len = tr_txt.str.len().values.astype(np.float64); te_len = te_txt.str.len().values.astype(np.float64)
tr_log1p = np.log1p(tr_len); te_log1p = np.log1p(te_len)
tr_age = train.get('requester_account_age_in_days_at_request', pd.Series([0]*ntr)).fillna(0).values.astype(np.float64)
te_age = test.get('requester_account_age_in_days_at_request', pd.Series([0]*nte)).fillna(0).values.astype(np.float64)
ts_tr = train[ts_col].values.astype(np.int64); ts_te = (test[ts_col].values.astype(np.int64) if ts_col in test.columns else np.zeros(nte, np.int64))
all_ts = np.concatenate([ts_tr, ts_te])
ord_all = np.argsort(all_ts)
rank_all = np.empty_like(ord_all)
rank_all[ord_all] = np.arange(len(all_ts))
rank01_all = rank_all.astype(np.float64) / max(1, (len(all_ts)-1))
tr_rank = rank01_all[:ntr]; te_rank = rank01_all[ntr:]

# Build feature matrices with interactions: base logits + metas + base*time_rank
def build_with_interactions(Z, log1p_len, age, trank):
    inter = Z * trank[:, None]
    return np.column_stack([Z, log1p_len, age, trank, inter]).astype(np.float64)
X_tr_full = build_with_interactions(Xb_tr, tr_log1p, tr_age, tr_rank)
X_te_full = build_with_interactions(Xb_te, te_log1p, te_age, te_rank)
feat_cols = names + ['log1p_text_len','account_age_days','time_rank01'] + [f'{n}*time' for n in names]
print('Final cols:', feat_cols[:min(10,len(feat_cols))], '... total', len(feat_cols))

# Forward-chaining OOF for C grid; evaluate AUC_last and gamma-weighted over blocks 2..5
val_blocks = [2,3,4,5]
gamma = 0.995
def gamma_weights_for_oof():
    W = np.zeros(ntr, dtype=np.float64)
    for bi in range(1, k):
        age = (k - 1) - bi
        w_block = (gamma ** age)
        if bi == 5: w_block *= 2.0
        W[blocks[bi]] = w_block
    return W
W_oof = gamma_weights_for_oof()
C_grid = [0.2, 0.5, 1.0, 2.0, 5.0]
best = None
for C in C_grid:
    oof = np.full(ntr, np.nan, dtype=np.float64)
    for vb in val_blocks:
        tr_idx = np.concatenate([blocks[i-1] for i in range(1, vb)])
        va_idx = blocks[vb-1]
        sc = StandardScaler(with_mean=True, with_std=True)
        Xtr = sc.fit_transform(X_tr_full[tr_idx])
        Xva = sc.transform(X_tr_full[va_idx])
        lr = LogisticRegression(penalty='l2', solver='lbfgs', C=C, max_iter=2000, fit_intercept=True)
        lr.fit(Xtr, y[tr_idx])
        z = lr.decision_function(Xva)
        p = sigmoid(z)
        oof[va_idx] = p
    mask = np.isfinite(oof)
    auc_last = roc_auc_score(y[blocks[4-1]], oof[blocks[4-1]])
    auc_gamma = roc_auc_score(y[mask], oof[mask], sample_weight=W_oof[mask])
    print(f'C={C} | AUC_last={auc_last:.5f} | AUC_gamma={auc_gamma:.5f}')
    if (best is None) or (auc_last > best['auc_last']) or (abs(auc_last - best['auc_last']) < 1e-12 and auc_gamma > best['auc_gamma']):
        best = dict(C=C, auc_last=auc_last, auc_gamma=auc_gamma)
print('Best C:', best)

# Final fit on blocks 1..4, evaluate on block 5 for sanity, then predict test
tr_final_idx = np.concatenate([blocks[i] for i in range(0,4)])
va_final_idx = blocks[4]
sc = StandardScaler(with_mean=True, with_std=True)
Xtr = sc.fit_transform(X_tr_full[tr_final_idx])
Xva = sc.transform(X_tr_full[va_final_idx])
Xte = sc.transform(X_te_full)
lr = LogisticRegression(penalty='l2', solver='lbfgs', C=best['C'], max_iter=2000, fit_intercept=True)
lr.fit(Xtr, y[tr_final_idx])
p_va = sigmoid(lr.decision_function(Xva))
auc_va = roc_auc_score(y[va_final_idx], p_va)
print(f'Final sanity AUC on block5={auc_va:.5f}')

p_gamma = sigmoid(lr.decision_function(Xte)).astype(np.float32)
pd.DataFrame({id_col: test[id_col].values, target_col: p_gamma}).to_csv('submission_s47lr_meta_gamma.csv', index=False)
print('Wrote submission_s47lr_meta_gamma.csv | mean', float(p_gamma.mean()))

# Safe test-time recency: interpolate selected base logits, rebuild features and predict
recent_map = {
    'LR_nosub': [
        'test_lr_time_nosub_meta_recent35.npy',
        'test_lr_time_nosub_meta_recent45.npy',
    ],
    'MiniLM': [
        'test_xgb_emb_meta_time_recent35.npy',
        'test_xgb_emb_meta_time_recent45.npy',
    ],
    'MPNet': [
        'test_xgb_emb_mpnet_time_recent35.npy',
        'test_xgb_emb_mpnet_time_recent45.npy',
    ],
}
name_to_j = {n:i for i,n in enumerate(names)}
def load_recent_avg_logit(files):
    arrs = []
    for fp in files:
        if os.path.exists(fp):
            a = np.load(fp)
            if a.ndim>1: a=a.ravel()
            arrs.append(to_logit(a))
    if not arrs: return None
    return np.mean(arrs, axis=0).astype(np.float64)

def apply_recency_to_Zte(Z_base, alphas):
    Zr = Z_base.copy()
    for bname, a in alphas.items():
        if a <= 0: continue
        if bname not in name_to_j: continue
        rec = load_recent_avg_logit(recent_map.get(bname, []))
        if rec is None: continue
        j = name_to_j[bname]
        Zr[:, j] = (1.0 - a)*Z_base[:, j] + a*rec
    return Zr

alphas_low = {'LR_nosub': 0.00, 'MiniLM': 0.15, 'MPNet': 0.20}
alphas_high = {'LR_nosub': 0.05, 'MiniLM': 0.25, 'MPNet': 0.30}

def predict_with_Zte(Zte_mod):
    Xte_mod = build_with_interactions(Zte_mod, te_log1p, te_age, te_rank)
    Xte_mod = sc.transform(Xte_mod)
    return sigmoid(lr.decision_function(Xte_mod)).astype(np.float32)

Zte_low = apply_recency_to_Zte(Xb_te, alphas_low)
Zte_high = apply_recency_to_Zte(Xb_te, alphas_high)
p_low = predict_with_Zte(Zte_low)
p_high = predict_with_Zte(Zte_high)
pd.DataFrame({id_col: test[id_col].values, target_col: p_low}).to_csv('submission_s47lr_meta_r_low.csv', index=False)
pd.DataFrame({id_col: test[id_col].values, target_col: p_high}).to_csv('submission_s47lr_meta_r_high.csv', index=False)
print('Wrote r_low/r_high | means ->', float(p_low.mean()), float(p_high.mean()))

# 3-way logit hedge and bias portfolio
def logit_clip(p, eps=1e-6):
    p = np.clip(p.astype(np.float64), eps, 1-eps)
    return np.log(p/(1-p))
z_g = logit_clip(p_gamma); z_l = logit_clip(p_low); z_h = logit_clip(p_high)
p_hedge3 = sigmoid((z_g + z_l + z_h)/3.0).astype(np.float32)
pd.DataFrame({id_col: test[id_col].values, target_col: p_hedge3}).to_csv('submission_s47lr_meta_hedge3.csv', index=False)
print('Wrote submission_s47lr_meta_hedge3.csv | mean', float(p_hedge3.mean()))

def bias_to_mean(probs, target, tol=1e-6, it=100):
    z = logit_clip(probs); lo, hi = -10.0, 10.0
    for _ in range(it):
        mid = 0.5*(lo+hi); m = sigmoid(z+mid).mean()
        if abs(m - target) < tol: return mid
        if m < target: lo = mid
        else: hi = mid
    return 0.5*(lo+hi)

for tm in [0.30, 0.32, 0.28]:
    b = bias_to_mean(p_hedge3, tm)
    pm = sigmoid(logit_clip(p_hedge3) + b).astype(np.float32)
    outp = f'submission_s47lr_meta_hedge3_m{int(round(tm*100)):03d}.csv'
    pd.DataFrame({id_col: test[id_col].values, target_col: pm}).to_csv(outp, index=False)
    print(f'Wrote {outp} | mean={pm.mean():.6f} | bias={b:.4f}')
    if abs(tm - 0.30) < 1e-9:
        pd.DataFrame({id_col: test[id_col].values, target_col: pm}).to_csv('submission.csv', index=False)
        print('PROMOTED: submission.csv <-', outp)

print(f'S47-LR-OOF done in {time.time()-t0:.1f}s')

In [ ]:
# S48-diagnostic: Validate alignment of base OOF arrays vs train indices; per-base AUCs by last block
import numpy as np, pandas as pd
from sklearn.metrics import roc_auc_score

print('S48: starting base OOF alignment diagnostic...')

# Reuse variables from latest run if present, else reload minimal artifacts
try:
    _names = names; _Z_oof_list = Z_oof_list; _blocks = blocks; _y = y
except NameError:
    train = pd.read_json('train.json')
    y = train['requester_received_pizza'].astype(int).values
    ts_col = 'unix_timestamp_of_request' if 'unix_timestamp_of_request' in train.columns else 'unix_timestamp_of_request_utc'
    order = np.argsort(train[ts_col].values)
    k = 6
    blocks = np.array_split(order, k)
    bases = [
        ('LR_nosub', 'oof_lr_time_nosub_meta.npy'),
        ('Dense_v1', 'oof_xgb_dense_time.npy'),
        ('Meta',    'oof_xgb_meta_time.npy'),
        ('MiniLM',  'oof_xgb_emb_meta_time.npy'),
        ('MPNet',   'oof_xgb_emb_mpnet_time.npy'),
        ('E5_meta', 'oof_xgb_e5_meta_time.npy'),
    ]
    def to_logit(p, eps=1e-6):
        p = np.clip(np.asarray(p, dtype=np.float64), eps, 1-eps)
        return np.log(p/(1-p))
    names = []
    Z_oof_list = []
    for name, fp in bases:
        if not os.path.exists(fp):
            print('Missing', fp, '-> skipping', name)
            continue
        a = np.load(fp); a = a.ravel()
        if a.shape[0] != len(train):
            print('Len mismatch for', name, a.shape[0], '!=', len(train))
            continue
        names.append(name); Z_oof_list.append(to_logit(a))
    _names, _Z_oof_list, _blocks, _y = names, Z_oof_list, blocks, y

n = len(_y)
last_block_idx = _blocks[4]  # block 5 indices (original indexing)

def safe_auc(y_true, y_score):
    try:
        return float(roc_auc_score(y_true, y_score))
    except Exception:
        return float('nan')

rows = []
for j, name in enumerate(_names):
    z = _Z_oof_list[j]  # logit-oof
    p = 1.0/(1.0+np.exp(-z))
    auc_overall = safe_auc(_y, p)
    auc_last = safe_auc(_y[last_block_idx], p[last_block_idx])
    # Alternate hypothesis: if OOF arrays were saved in time-sorted order already, the last block would correspond to the tail slice
    tail_len = len(last_block_idx)
    tail_idx = np.arange(n - tail_len, n)
    auc_tail = safe_auc(_y[tail_idx], p[tail_idx])
    rows.append((name, auc_overall, auc_last, auc_tail))

df = pd.DataFrame(rows, columns=['Base','AUC_overall','AUC_last(block5 by idx)','AUC_tail(last N rows)'])
print(df.to_string(index=False))
df.to_csv('s48_base_oof_alignment.csv', index=False)
print('Saved s48_base_oof_alignment.csv')

In [ ]:
# S49-LR_STRONG: Time-aware TFIDF(word1-2 + char_wb3-6) + light metas -> LR with forward-chaining OOF
import os, time
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

t0 = time.time()
id_col='request_id'; ycol='requester_received_pizza'; ts='unix_timestamp_of_request'
tr=pd.read_json('train.json'); te=pd.read_json('test.json')
y=tr[ycol].astype(int).values; n=len(tr); m=len(te)
ts_col = ts if ts in tr.columns else 'unix_timestamp_of_request_utc'
order=np.argsort(tr[ts_col].values); k=6; blocks=np.array_split(order,k)

def get_text(df):
    t=df.get('request_title', pd.Series(['']*len(df))).fillna('').astype(str)
    b=df.get('request_text_edit_aware', df.get('request_text', pd.Series(['']*len(df)))).fillna('').astype(str)
    return t+'\n'+b
tx_tr=get_text(tr); tx_te=get_text(te)

# lightweight metas present in BOTH train/test
meta_cands=['requester_account_age_in_days_at_request','requester_number_of_posts_at_request','requester_number_of_comments_at_request','requester_upvotes_minus_downvotes_at_request','requester_upvotes_plus_downvotes_at_request']
meta_cols=[c for c in meta_cands if (c in tr.columns and c in te.columns)]
log1p_len_tr=np.log1p(tx_tr.str.len().values)
log1p_len_te=np.log1p(tx_te.str.len().values)
Xmeta_tr=np.column_stack([log1p_len_tr]+[tr[c].fillna(0).astype(float).values for c in meta_cols])
Xmeta_te=np.column_stack([log1p_len_te]+[te[c].fillna(0).astype(float).values for c in meta_cols])

# Vectorizers
v_w=TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=2, max_df=0.95, max_features=400000, strip_accents='unicode', lowercase=True, sublinear_tf=True)
v_c=TfidfVectorizer(analyzer='char_wb', ngram_range=(3,6), min_df=2, max_features=400000, sublinear_tf=True)
Xw_tr=v_w.fit_transform(tx_tr); Xw_te=v_w.transform(tx_te)
Xc_tr=v_c.fit_transform(tx_tr); Xc_te=v_c.transform(tx_te)
Xtxt_tr=sparse.hstack([Xw_tr,Xc_tr],format='csr')
Xtxt_te=sparse.hstack([Xw_te,Xc_te],format='csr')
X_tr=sparse.hstack([Xtxt_tr, sparse.csr_matrix(Xmeta_tr)],format='csr')
X_te=sparse.hstack([Xtxt_te, sparse.csr_matrix(Xmeta_te)],format='csr')
print('Shapes -> X_tr:', X_tr.shape, 'X_te:', X_te.shape, '| metas used:', meta_cols)

def fit_lr(C):
    return LogisticRegression(solver='liblinear', penalty='l2', C=C, max_iter=3000, random_state=42)

best=None
C_list=[1.0,2.0,3.0,5.0]
for C in C_list:
    oof=np.full(n, np.nan, float)
    for vb in [2,3,4,5]:
        tr_idx=np.concatenate([blocks[i-1] for i in range(1,vb)])
        va_idx=blocks[vb-1]
        lr=fit_lr(C); lr.fit(X_tr[tr_idx], y[tr_idx])
        oof[va_idx]=lr.predict_proba(X_tr[va_idx])[:,1]
    auc_last=roc_auc_score(y[blocks[4-1]], oof[blocks[4-1]])
    print(f'C={C} | AUC_last={auc_last:.5f}')
    if (best is None) or (auc_last>best[0]): best=(auc_last,C,oof)
print('BestC:', best[1], '| AUC_last=', f'{best[0]:.5f}')

# Final fit on blocks 1..4 and predict test
tr_final=np.concatenate([blocks[i] for i in range(0,4)])
lr=fit_lr(best[1]); lr.fit(X_tr[tr_final], y[tr_final])
p_te=lr.predict_proba(X_te)[:,1].astype(np.float32)
np.save('oof_lr_wordchar_meta_time.npy', best[2].astype(np.float32))
np.save('test_lr_wordchar_meta_time.npy', p_te)
print('Saved oof_lr_wordchar_meta_time.npy / test_lr_wordchar_meta_time.npy | test mean=', float(p_te.mean()), '| took', f'{time.time()-t0:.1f}s')

In [ ]:
# S49b-LR_STRONG_OOF_FOLDS: Per-fold vectorizers (no leakage) + richer metas -> TFIDF LR strong base
import re, time, os
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

t0 = time.time()
id_col='request_id'; ycol='requester_received_pizza'; ts='unix_timestamp_of_request'
tr=pd.read_json('train.json'); te=pd.read_json('test.json')
y=tr[ycol].astype(int).values; n=len(tr); m=len(te)
ts_col = ts if ts in tr.columns else 'unix_timestamp_of_request_utc'
order=np.argsort(tr[ts_col].values); k=6; blocks=np.array_split(order,k)

def get_text(df):
    t=df.get('request_title', pd.Series(['']*len(df))).fillna('').astype(str)
    b=df.get('request_text_edit_aware', df.get('request_text', pd.Series(['']*len(df)))).fillna('').astype(str)
    return t+'\n'+b
tx_tr=get_text(tr); tx_te=get_text(te)

# Lightweight engineered metas (present in both) + text-derived flags/counts
def has_link(s): return 1 if re.search(r'(http|www\.)', s, re.I) else 0
def has_image(s): return 1 if re.search(r'\.(jpg|jpeg|png|gif)\b', s, re.I) else 0
money_re = re.compile(r'(\$|dollar|rent|bill|pay|cash)', re.I)
urgency_re = re.compile(r'(urgent|emergency|asap|today|tonight)', re.I)
def count_pat(p, s):
    m = p.findall(s)
    return len(m) if m else 0

def build_meta(df, tx):
    cols = []
    # numeric requester stats if present
    base_feats = ['requester_account_age_in_days_at_request','requester_number_of_posts_at_request','requester_number_of_comments_at_request','requester_upvotes_minus_downvotes_at_request','requester_upvotes_plus_downvotes_at_request']
    arrs = []
    for c in base_feats:
        if c in df.columns: arrs.append(df[c].fillna(0).astype(float).values); cols.append(c)
    # text length
    log1p_len = np.log1p(tx.str.len().values); arrs.append(log1p_len); cols.append('log1p_text_len')
    # binary/counts
    link_f = np.array([has_link(s) for s in tx], dtype=np.float32); arrs.append(link_f); cols.append('has_link')
    img_f = np.array([has_image(s) for s in tx], dtype=np.float32); arrs.append(img_f); cols.append('has_image')
    money_c = np.array([count_pat(money_re, s) for s in tx], dtype=np.float32); arrs.append(money_c); cols.append('money_cnt')
    urg_c = np.array([count_pat(urgency_re, s) for s in tx], dtype=np.float32); arrs.append(urg_c); cols.append('urgency_cnt')
    excl_c = np.array([s.count('!') for s in tx], dtype=np.float32); arrs.append(excl_c); cols.append('exclam_cnt')
    return np.column_stack(arrs).astype(np.float32), cols

Xmeta_tr, meta_cols = build_meta(tr, tx_tr)
Xmeta_te, _ = build_meta(te, tx_te)

# Vectorizer params
w_params=dict(analyzer='word', ngram_range=(1,2), min_df=2, max_df=0.95, max_features=300000, strip_accents='unicode', lowercase=True, sublinear_tf=True)
c_params=dict(analyzer='char_wb', ngram_range=(3,6), min_df=2, max_features=300000, sublinear_tf=True)

def fit_lr(C):
    return LogisticRegression(solver='liblinear', penalty='l2', C=C, max_iter=3000, random_state=42)

C_list=[1.0, 2.0, 3.0, 5.0]
best=None
for C in C_list:
    oof=np.full(n, np.nan, dtype=np.float32)
    for vb in [2,3,4,5]:
        tr_idx=np.concatenate([blocks[i-1] for i in range(1,vb)])
        va_idx=blocks[vb-1]
        # Fit vectorizers on training folds only
        vw=TfidfVectorizer(**w_params); vc=TfidfVectorizer(**c_params)
        Xw_tr = vw.fit_transform(tx_tr.iloc[tr_idx]); Xw_va = vw.transform(tx_tr.iloc[va_idx])
        Xc_tr = vc.fit_transform(tx_tr.iloc[tr_idx]); Xc_va = vc.transform(tx_tr.iloc[va_idx])
        Xtr_txt = sparse.hstack([Xw_tr, Xc_tr], format='csr')
        Xva_txt = sparse.hstack([Xw_va, Xc_va], format='csr')
        Xtr = sparse.hstack([Xtr_txt, sparse.csr_matrix(Xmeta_tr[tr_idx])], format='csr')
        Xva = sparse.hstack([Xva_txt, sparse.csr_matrix(Xmeta_tr[va_idx])], format='csr')
        lr=fit_lr(C); lr.fit(Xtr, y[tr_idx])
        oof[va_idx] = lr.predict_proba(Xva)[:,1].astype(np.float32)
    auc_last=roc_auc_score(y[blocks[4-1]], oof[blocks[4-1]])
    print(f'[Leak-free] C={C} | AUC_last={auc_last:.5f}')
    if (best is None) or (auc_last>best[0]): best=(auc_last,C,oof)
print('BestC (leak-free OOF):', best[1], '| AUC_last=', f'{best[0]:.5f}')

# Final fit: train on blocks 1..4 with vectorizers fit on those texts, then predict test
tr_final=np.concatenate([blocks[i] for i in range(0,4)])
vw=TfidfVectorizer(**w_params); vc=TfidfVectorizer(**c_params)
Xw_tr_f = vw.fit_transform(tx_tr.iloc[tr_final]); Xw_te_f = vw.transform(tx_te)
Xc_tr_f = vc.fit_transform(tx_tr.iloc[tr_final]); Xc_te_f = vc.transform(tx_te)
Xtr_txt_f = sparse.hstack([Xw_tr_f, Xc_tr_f], format='csr')
Xte_txt_f = sparse.hstack([Xw_te_f, Xc_te_f], format='csr')
Xtr_f = sparse.hstack([Xtr_txt_f, sparse.csr_matrix(Xmeta_tr[tr_final])], format='csr')
Xte_f = sparse.hstack([Xte_txt_f, sparse.csr_matrix(Xmeta_te)], format='csr')
lr=fit_lr(best[1]); lr.fit(Xtr_f, y[tr_final])
p_te=lr.predict_proba(Xte_f)[:,1].astype(np.float32)

# Save artifacts
np.save('oof_lr_wordchar_meta_time.npy', best[2].astype(np.float32))
np.save('test_lr_wordchar_meta_time.npy', p_te)
print('Saved oof/test for LR_STRONG | test mean=', float(p_te.mean()), '| total time', f'{time.time()-t0:.1f}s')

In [5]:
# S50-CA-BLEND: Coordinate-ascent logit blend on block 5, recency hedges (with caps/floors), bias to 0.30/0.32
import os, numpy as np, pandas as pd, time
from sklearn.metrics import roc_auc_score

t0=time.time()
id_col='request_id'; target_col='requester_received_pizza'
train=pd.read_json('train.json'); test=pd.read_json('test.json')
y=train[target_col].astype(int).values; ids=test[id_col].values

def to_logit(p, eps=1e-6):
    p=np.clip(p.astype(np.float64), eps, 1-eps); return np.log(p/(1-p))
def sigmoid(z): return 1.0/(1.0+np.exp(-z))

# Time blocks and last validated block (block 5 = index 4)
ts_col='unix_timestamp_of_request' if 'unix_timestamp_of_request' in train.columns else 'unix_timestamp_of_request_utc'
order=np.argsort(train[ts_col].values); k=6; blocks=np.array_split(order,k)
last_idx = blocks[4]

# Candidate bases (OOF/test probs). Prefer tuned LR_strong if available.
cands=[
 ('LR_nosub','oof_lr_time_nosub_meta.npy','test_lr_time_nosub_meta.npy'),
 ('Dense_v1','oof_xgb_dense_time.npy','test_xgb_dense_time.npy'),
 ('Meta','oof_xgb_meta_time.npy','test_xgb_meta_time.npy'),
 ('MiniLM','oof_xgb_emb_meta_time.npy','test_xgb_emb_meta_time.npy'),
 ('MPNet','oof_xgb_emb_mpnet_time.npy','test_xgb_emb_mpnet_time.npy'),
 ('CatBoost_v2','oof_catboost_textmeta_v2.npy','test_catboost_textmeta_v2.npy'),
 # LR_strong tuned fallback
 ('LR_strong_tuned','oof_lr_wordchar_meta_time_tuned.npy','test_lr_wordchar_meta_time_tuned.npy'),
 ('LR_strong','oof_lr_wordchar_meta_time.npy','test_lr_wordchar_meta_time.npy'),
]

def pick_lr_strong(entries):
    # prefer tuned over untuned
    prefer = None; fallback = None
    for nm,oo,tt in entries:
        if nm=='LR_strong_tuned' and os.path.exists(oo) and os.path.exists(tt):
            prefer = ('LR_strong', oo, tt)  # normalize name to LR_strong
        if nm=='LR_strong' and os.path.exists(oo) and os.path.exists(tt):
            fallback = ('LR_strong', oo, tt)
    return prefer if prefer is not None else fallback

OOF_list=[]; TEST_list=[]; names=[]; auc_last_rows=[]
tmp=[]
for nm,oo,tt in cands:
    if nm.startswith('LR_strong'):
        tmp.append((nm,oo,tt))
        continue
    if os.path.exists(oo) and os.path.exists(tt):
        o=np.load(oo).astype(np.float64).ravel(); t=np.load(tt).astype(np.float64).ravel()
        if len(o)==len(y):
            auc_last = roc_auc_score(y[last_idx], o[last_idx])
            auc_last_rows.append((nm, auc_last))
            OOF_list.append(to_logit(o)); TEST_list.append(to_logit(t)); names.append(nm)

# handle LR_strong choice
lr_choice = pick_lr_strong(tmp)
if lr_choice is not None:
    nm, oo, tt = lr_choice
    o=np.load(oo).astype(np.float64).ravel(); t=np.load(tt).astype(np.float64).ravel()
    if len(o)==len(y):
        auc_last = roc_auc_score(y[last_idx], o[last_idx])
        auc_last_rows.append((nm, auc_last))
        OOF_list.append(to_logit(o)); TEST_list.append(to_logit(t)); names.append(nm)

print('Loaded bases with AUC_last:', sorted(auc_last_rows, key=lambda x: -x[1]))

if len(names)<3:
    raise RuntimeError('Too few bases loaded for blending.')

# Prune by last-block AUC >= 0.60 (keep at least top-5 if needed)
by_name_auc = {nm:auc for nm,auc in auc_last_rows}
keep = [nm for nm in names if by_name_auc.get(nm, 0.0) >= 0.60]
if len(keep) < 5:
    # fallback: keep top-5 by AUC_last
    sorted_by = sorted([(nm, OOF_list[i], TEST_list[i], by_name_auc.get(nm, 0.0)) for i,nm in enumerate(names)], key=lambda x: -x[3])[:5]
    names = [x[0] for x in sorted_by]
    OOF_list = [x[1] for x in sorted_by]
    TEST_list = [x[2] for x in sorted_by]
else:
    names_keep=[]; O_keep=[]; T_keep=[]
    for i,nm in enumerate(names):
        if nm in keep:
            names_keep.append(nm); O_keep.append(OOF_list[i]); T_keep.append(TEST_list[i])
    names, OOF_list, TEST_list = names_keep, O_keep, T_keep
print('Bases kept:', names)

OOF = np.column_stack(OOF_list)  # (n,k)
TEST = np.column_stack(TEST_list) # (m,k)
k_b = OOF.shape[1]

def auc_last_w(w):
    z = OOF @ w
    return roc_auc_score(y[last_idx], z[last_idx])

# Initialize weights
w0 = np.ones(k_b, dtype=np.float64) / k_b
init_map = {'LR_strong':0.36,'Dense_v1':0.14,'Meta':0.18,'MiniLM':0.16,'MPNet':0.16,'LR_nosub':0.0,'CatBoost_v2':0.05}
w = np.array([init_map.get(n, 0.0) for n in names], dtype=np.float64)
if w.sum() <= 0: w = w0.copy()
else: w = w / w.sum()
base_auc = auc_last_w(w)
print('Init AUC_last=', f'{base_auc:.5f}', '| init w=', dict(zip(names, np.round(w,3))))

# Caps and floors per expert advice
caps = {
    'CatBoost_v2': 0.50,
    'LR_strong': 0.60,
    'MiniLM': 0.40,
    'MPNet': 0.40,
    'Dense_v1': 0.30,
    'Meta': 0.30,
    'LR_nosub': 0.20,
}
global_cap = 0.55
floors = {
    'LR_strong': 0.18,
    'MiniLM': 0.10,
    'MPNet': 0.10,
    'Dense_v1': 0.05,
    'Meta': 0.05,
}
name_to_j = {n:i for i,n in enumerate(names)}

def violates_caps(v):
    # per-base caps and global single-base cap
    if np.max(v) > global_cap + 1e-12: return True
    for nm, cap in caps.items():
        if nm in name_to_j and v[name_to_j[nm]] > cap + 1e-12:
            return True
    return False

def apply_floors(v):
    v2 = v.copy()
    for nm, fl in floors.items():
        if nm in name_to_j:
            j = name_to_j[nm]
            if v2[j] < fl: v2[j] = fl
    v2 = v2 / v2.sum()
    return v2

# Coordinate ascent with nonnegativity, sum=1, caps, and floors enforcement after each pass
grid = np.linspace(0.0, 1.0, 21)
for it in range(8):
    improved=False
    for j in range(k_b):
        best_auc=base_auc; best_a = w[j]; best_w = w.copy()
        for a in grid:
            v = w.copy()
            # set weight j = a, renormalize others to sum to 1-a
            v_others = v.copy(); v_others[j]=0.0
            if v_others.sum()>0:
                v_others = (1.0 - a) * v_others / v_others.sum()
            else:
                v_others = np.zeros_like(v_others)
            v = v_others; v[j]=a
            if violates_caps(v):
                continue
            A = auc_last_w(v)
            if A > best_auc + 1e-6:
                best_auc, best_a, best_w = A, a, v.copy()
        if best_auc > base_auc + 1e-6:
            w = best_w; base_auc = best_auc; improved=True
    # apply floors after each full pass and recompute auc
    w = apply_floors(w)
    base_auc = auc_last_w(w)
    print(f'Iter {it}: AUC_last={base_auc:.5f} | w=', dict(zip(names, np.round(w,3))))
    if not improved: break

print('Final AUC_last:', f'{base_auc:.5f}', '| weights:', dict(zip(names, np.round(w,4))))

# Build gamma (no recency) test probs
z_te = TEST @ w
p_gamma = sigmoid(z_te).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p_gamma}).to_csv('submission_s50_ca_gamma.csv', index=False)
print('Wrote submission_s50_ca_gamma.csv | mean', float(p_gamma.mean()))

# Recency on embeddings only (MiniLM, MPNet); add tiny LR_nosub in high per advice
def load_recent_avg(prefixes):
    arr=[]
    for fp in prefixes:
        if os.path.exists(fp):
            a=np.load(fp); a=a.ravel().astype(np.float64); arr.append(a)
    return np.mean(arr,axis=0) if arr else None

def apply_recency_to_TEST(TEST_base, alphas):
    Z = TEST_base.copy()
    # LR_nosub
    if 'LR_nosub' in name_to_j and alphas.get('LR_nosub',0)>0:
        pr = load_recent_avg(['test_lr_time_nosub_meta_recent35.npy','test_lr_time_nosub_meta_recent45.npy'])
        if pr is not None:
            j = name_to_j['LR_nosub']
            Z[:,j] = (1.0 - alphas['LR_nosub'])*Z[:,j] + alphas['LR_nosub']*to_logit(pr)
    # MiniLM
    if 'MiniLM' in name_to_j and alphas.get('MiniLM',0)>0:
        pr = load_recent_avg(['test_xgb_emb_meta_time_recent35.npy','test_xgb_emb_meta_time_recent45.npy'])
        if pr is not None:
            j = name_to_j['MiniLM']
            Z[:,j] = (1.0 - alphas['MiniLM'])*Z[:,j] + alphas['MiniLM']*to_logit(pr)
    # MPNet
    if 'MPNet' in name_to_j and alphas.get('MPNet',0)>0:
        pr = load_recent_avg(['test_xgb_emb_mpnet_time_recent35.npy','test_xgb_emb_mpnet_time_recent45.npy'])
        if pr is not None:
            j = name_to_j['MPNet']
            Z[:,j] = (1.0 - alphas['MPNet'])*Z[:,j] + alphas['MPNet']*to_logit(pr)
    return Z

# Updated alphas per expert advice
alphas_low = {'MiniLM': 0.18, 'MPNet': 0.22}
alphas_high= {'MiniLM': 0.28, 'MPNet': 0.32, 'LR_nosub': 0.05}
TEST_low  = apply_recency_to_TEST(TEST, alphas_low)
TEST_high = apply_recency_to_TEST(TEST, alphas_high)
p_low  = sigmoid(TEST_low @ w).astype(np.float32)
p_high = sigmoid(TEST_high @ w).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p_low}).to_csv('submission_s50_ca_r_low.csv', index=False)
pd.DataFrame({id_col: ids, target_col: p_high}).to_csv('submission_s50_ca_r_high.csv', index=False)
print('Wrote r_low/r_high means ->', float(p_low.mean()), float(p_high.mean()))

# 3-way logit hedge: gamma, r_low, r_high
zg, zl, zh = to_logit(p_gamma), to_logit(p_low), to_logit(p_high)
p3 = sigmoid((zg+zl+zh)/3.0).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p3}).to_csv('submission_s50_ca_hedge3.csv', index=False)
print('Wrote submission_s50_ca_hedge3.csv | mean', float(p3.mean()))

# Bias to 0.30 and 0.32; promote 0.30
def bias_to_mean(probs, target, it=80):
    z = to_logit(probs); lo,hi=-10.0,10.0
    for _ in range(it):
        mid=(lo+hi)/2; m=sigmoid(z+mid).mean()
        if m<target: lo=mid
        else: hi=mid
    return (lo+hi)/2

for tm in [0.30, 0.32]:
    b = bias_to_mean(p3, tm)
    pm = sigmoid(to_logit(p3)+b).astype(np.float32)
    outp = f'submission_s50_ca_hedge3_m{int(tm*100):03d}.csv'
    pd.DataFrame({id_col: ids, target_col: pm}).to_csv(outp, index=False)
    print(f'Wrote {outp} | mean={pm.mean():.6f}')
    if abs(tm-0.30)<1e-9:
        pd.DataFrame({id_col: ids, target_col: pm}).to_csv('submission.csv', index=False)
        print('PROMOTED: submission.csv <-', outp)

print('S50-CA-BLEND done in', f'{time.time()-t0:.1f}s')

Loaded bases with AUC_last: [('CatBoost_v2', 0.650664850639457), ('MiniLM', 0.6444665035320191), ('Dense_v1', 0.6396417193776259), ('MPNet', 0.6346091693984025), ('LR_nosub', 0.6235744955907474), ('Meta', 0.621023592963664), ('LR_strong', 0.6190498176277761)]
Bases kept: ['LR_nosub', 'Dense_v1', 'Meta', 'MiniLM', 'MPNet', 'CatBoost_v2', 'LR_strong']
Init AUC_last= 0.64617 | init w= {'LR_nosub': 0.0, 'Dense_v1': 0.133, 'Meta': 0.171, 'MiniLM': 0.152, 'MPNet': 0.152, 'CatBoost_v2': 0.048, 'LR_strong': 0.343}
Iter 0: AUC_last=0.65398 | w= {'LR_nosub': 0.016, 'Dense_v1': 0.048, 'Meta': 0.072, 'MiniLM': 0.145, 'MPNet': 0.113, 'CatBoost_v2': 0.445, 'LR_strong': 0.16}


Iter 1: AUC_last=0.65564 | w= {'LR_nosub': 0.0, 'Dense_v1': 0.067, 'Meta': 0.044, 'MiniLM': 0.205, 'MPNet': 0.088, 'CatBoost_v2': 0.438, 'LR_strong': 0.158}


Iter 2: AUC_last=0.65562 | w= {'LR_nosub': 0.0, 'Dense_v1': 0.065, 'Meta': 0.043, 'MiniLM': 0.143, 'MPNet': 0.165, 'CatBoost_v2': 0.429, 'LR_strong': 0.155}


Iter 3: AUC_last=0.65550 | w= {'LR_nosub': 0.0, 'Dense_v1': 0.044, 'Meta': 0.044, 'MiniLM': 0.174, 'MPNet': 0.147, 'CatBoost_v2': 0.435, 'LR_strong': 0.157}
Iter 4: AUC_last=0.65569 | w= {'LR_nosub': 0.0, 'Dense_v1': 0.053, 'Meta': 0.044, 'MiniLM': 0.215, 'MPNet': 0.088, 'CatBoost_v2': 0.441, 'LR_strong': 0.159}


Iter 5: AUC_last=0.65543 | w= {'LR_nosub': 0.0, 'Dense_v1': 0.064, 'Meta': 0.043, 'MiniLM': 0.143, 'MPNet': 0.165, 'CatBoost_v2': 0.43, 'LR_strong': 0.155}
Iter 6: AUC_last=0.65585 | w= {'LR_nosub': 0.0, 'Dense_v1': 0.055, 'Meta': 0.043, 'MiniLM': 0.23, 'MPNet': 0.086, 'CatBoost_v2': 0.431, 'LR_strong': 0.155}


Iter 7: AUC_last=0.65548 | w= {'LR_nosub': 0.0, 'Dense_v1': 0.09, 'Meta': 0.043, 'MiniLM': 0.198, 'MPNet': 0.086, 'CatBoost_v2': 0.429, 'LR_strong': 0.154}
Final AUC_last: 0.65548 | weights: {'LR_nosub': 0.0, 'Dense_v1': 0.0898, 'Meta': 0.0429, 'MiniLM': 0.1982, 'MPNet': 0.0858, 'CatBoost_v2': 0.4289, 'LR_strong': 0.1544}
Wrote submission_s50_ca_gamma.csv | mean 0.33813750743865967
Wrote r_low/r_high means -> 0.3257971704006195 0.319436252117157
Wrote submission_s50_ca_hedge3.csv | mean 0.3277496099472046
Wrote submission_s50_ca_hedge3_m030.csv | mean=0.300000
PROMOTED: submission.csv <- submission_s50_ca_hedge3_m030.csv
Wrote submission_s50_ca_hedge3_m032.csv | mean=0.320000
S50-CA-BLEND done in 1.8s


In [4]:
# S49c-LR_STRONG_TUNED: Per-fold TFIDF word(1-3)+char_wb(3-5), min_df=1, larger vocab, class_weight variants
import re, time, os
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

t0 = time.time()
id_col='request_id'; ycol='requester_received_pizza'; ts='unix_timestamp_of_request'
tr=pd.read_json('train.json'); te=pd.read_json('test.json')
y=tr[ycol].astype(int).values; n=len(tr); m=len(te)
ts_col = ts if ts in tr.columns else 'unix_timestamp_of_request_utc'
order=np.argsort(tr[ts_col].values); k=6; blocks=np.array_split(order,k)

def get_text(df):
    t=df.get('request_title', pd.Series(['']*len(df))).fillna('').astype(str)
    b=df.get('request_text_edit_aware', df.get('request_text', pd.Series(['']*len(df)))).fillna('').astype(str)
    return t+'\n'+b
tx_tr=get_text(tr); tx_te=get_text(te)

# Lightweight engineered metas (numeric present in both) + simple text flags/counts
def has_link(s): return 1 if re.search(r'(http|www\.)', s, re.I) else 0
def has_image(s): return 1 if re.search(r'\.(jpg|jpeg|png|gif)\b', s, re.I) else 0
money_re = re.compile(r'(\$|dollar|rent|bill|pay|cash)', re.I)
urgency_re = re.compile(r'(urgent|emergency|asap|today|tonight)', re.I)
def cnt(p, s):
    m = p.findall(s); return len(m) if m else 0

def build_meta(df, tx):
    cols = []; arrs = []
    base = ['requester_account_age_in_days_at_request','requester_number_of_posts_at_request','requester_number_of_comments_at_request','requester_upvotes_minus_downvotes_at_request','requester_upvotes_plus_downvotes_at_request']
    for c in base:
        if c in df.columns: arrs.append(df[c].fillna(0).astype(float).values); cols.append(c)
    log1p_len = np.log1p(tx.str.len().values); arrs.append(log1p_len); cols.append('log1p_text_len')
    link_f = np.array([has_link(s) for s in tx], dtype=np.float32); arrs.append(link_f); cols.append('has_link')
    img_f = np.array([has_image(s) for s in tx], dtype=np.float32); arrs.append(img_f); cols.append('has_image')
    money_c = np.array([cnt(money_re, s) for s in tx], dtype=np.float32); arrs.append(money_c); cols.append('money_cnt')
    urg_c = np.array([cnt(urgency_re, s) for s in tx], dtype=np.float32); arrs.append(urg_c); cols.append('urgency_cnt')
    excl_c = np.array([s.count('!') for s in tx], dtype=np.float32); arrs.append(excl_c); cols.append('exclam_cnt')
    return np.column_stack(arrs).astype(np.float32), cols

Xmeta_tr, meta_cols = build_meta(tr, tx_tr)
Xmeta_te, _ = build_meta(te, tx_te)

# Tuned vectorizers
w_params=dict(analyzer='word', ngram_range=(1,3), min_df=1, max_df=0.98, max_features=600000, strip_accents='unicode', lowercase=True, sublinear_tf=True)
c_params=dict(analyzer='char_wb', ngram_range=(3,5), min_df=1, max_features=400000, sublinear_tf=True)

def fit_lr(C, cw=None):
    return LogisticRegression(solver='liblinear', penalty='l2', C=C, max_iter=4000, random_state=42, class_weight=cw)

C_list=[1.0, 2.0, 3.0, 5.0, 8.0, 12.0]
cw_list=[None, 'balanced']
best=None
for cw in cw_list:
    for C in C_list:
        oof=np.full(n, np.nan, dtype=np.float32)
        for vb in [2,3,4,5]:
            tr_idx=np.concatenate([blocks[i-1] for i in range(1,vb)])
            va_idx=blocks[vb-1]
            vw=TfidfVectorizer(**w_params); vc=TfidfVectorizer(**c_params)
            Xw_tr = vw.fit_transform(tx_tr.iloc[tr_idx]); Xw_va = vw.transform(tx_tr.iloc[va_idx])
            Xc_tr = vc.fit_transform(tx_tr.iloc[tr_idx]); Xc_va = vc.transform(tx_tr.iloc[va_idx])
            Xtr_txt = sparse.hstack([Xw_tr, Xc_tr], format='csr')
            Xva_txt = sparse.hstack([Xw_va, Xc_va], format='csr')
            Xtr = sparse.hstack([Xtr_txt, sparse.csr_matrix(Xmeta_tr[tr_idx])], format='csr')
            Xva = sparse.hstack([Xva_txt, sparse.csr_matrix(Xmeta_tr[va_idx])], format='csr')
            lr=fit_lr(C, cw); lr.fit(Xtr, y[tr_idx])
            oof[va_idx] = lr.predict_proba(Xva)[:,1].astype(np.float32)
        # Evaluate on true last validated block (block index 4)
        auc_last=roc_auc_score(y[blocks[4]], oof[blocks[4]])
        print(f'[Leak-free] cw={cw} C={C} | AUC_last={auc_last:.5f}')
        if (best is None) or (auc_last>best[0]): best=(auc_last,(C,cw),oof)
print('Best params (leak-free OOF):', best[1], '| AUC_last=', f'{best[0]:.5f}')

# Final fit on blocks 1..4 with tuned vectorizers and best params; predict test
tr_final=np.concatenate([blocks[i] for i in range(0,4)])
vw=TfidfVectorizer(**w_params); vc=TfidfVectorizer(**c_params)
Xw_tr_f = vw.fit_transform(tx_tr.iloc[tr_final]); Xw_te_f = vw.transform(tx_te)
Xc_tr_f = vc.fit_transform(tx_tr.iloc[tr_final]); Xc_te_f = vc.transform(tx_te)
Xtr_txt_f = sparse.hstack([Xw_tr_f, Xc_tr_f], format='csr')
Xte_txt_f = sparse.hstack([Xw_te_f, Xc_te_f], format='csr')
Xtr_f = sparse.hstack([Xtr_txt_f, sparse.csr_matrix(Xmeta_tr[tr_final])], format='csr')
Xte_f = sparse.hstack([Xte_txt_f, sparse.csr_matrix(Xmeta_te)], format='csr')
C_best, cw_best = best[1]
lr=fit_lr(C_best, cw_best); lr.fit(Xtr_f, y[tr_final])
p_te=lr.predict_proba(Xte_f)[:,1].astype(np.float32)

# Save artifacts for blending
np.save('oof_lr_wordchar_meta_time_tuned.npy', best[2].astype(np.float32))
np.save('test_lr_wordchar_meta_time_tuned.npy', p_te)
print('Saved tuned LR_STRONG OOF/test | test mean=', float(p_te.mean()), '| total time', f'{time.time()-t0:.1f}s')

[Leak-free] cw=None C=1.0 | AUC_last=0.61573


[Leak-free] cw=None C=2.0 | AUC_last=0.61589


[Leak-free] cw=None C=3.0 | AUC_last=0.61166


[Leak-free] cw=None C=5.0 | AUC_last=0.60797


[Leak-free] cw=None C=8.0 | AUC_last=0.60963


...

[Leak-free] cw=balanced C=3.0 | AUC_last=0.61547


[Leak-free] cw=balanced C=5.0 | AUC_last=0.61106


[Leak-free] cw=balanced C=8.0 | AUC_last=0.61169


[Leak-free] cw=balanced C=12.0 | AUC_last=0.60635
Best params (leak-free OOF): (2.0, 'balanced') | AUC_last= 0.61905


Saved tuned LR_STRONG OOF/test | test mean= 0.39611732959747314 | total time 258.1s


In [ ]:
# S51-LASTBLOCK-LR: Fit LR stacker on last block only (base logits), recency hedges, 3-way logit hedge, bias to 0.30/0.32
import os, numpy as np, pandas as pd, time
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

t0=time.time()
id_col='request_id'; target_col='requester_received_pizza'
train=pd.read_json('train.json'); test=pd.read_json('test.json')
y=train[target_col].astype(int).values; ids=test[id_col].values

def to_logit(p, eps=1e-6):
    p=np.clip(p.astype(np.float64), eps, 1-eps); return np.log(p/(1-p))
def sigmoid(z): return 1.0/(1.0+np.exp(-z))

# Time blocks and last validated block (block 5 = index 4)
ts_col='unix_timestamp_of_request' if 'unix_timestamp_of_request' in train.columns else 'unix_timestamp_of_request_utc'
order=np.argsort(train[ts_col].values); k=6; blocks=np.array_split(order,k)
last_idx = blocks[4]

# Bases to use (OOF/test probs). Prefer diverse, pruned set.
bases=[
 ('LR_nosub','oof_lr_time_nosub_meta.npy','test_lr_time_nosub_meta.npy'),
 ('Dense_v1','oof_xgb_dense_time.npy','test_xgb_dense_time.npy'),
 ('Meta','oof_xgb_meta_time.npy','test_xgb_meta_time.npy'),
 ('MiniLM','oof_xgb_emb_meta_time.npy','test_xgb_emb_meta_time.npy'),
 ('MPNet','oof_xgb_emb_mpnet_time.npy','test_xgb_emb_mpnet_time.npy'),
 ('CatBoost_v2','oof_catboost_textmeta_v2.npy','test_catboost_textmeta_v2.npy'),
]
OOF_list=[]; TEST_list=[]; names=[]; auc_rows=[]
for nm,oo,tt in bases:
    if os.path.exists(oo) and os.path.exists(tt):
        o=np.load(oo).astype(np.float64).ravel(); t=np.load(tt).astype(np.float64).ravel()
        if len(o)==len(y):
            OOF_list.append(to_logit(o)); TEST_list.append(to_logit(t)); names.append(nm)
            try:
                auc_rows.append((nm, float(roc_auc_score(y[last_idx], o[last_idx]))))
            except Exception:
                auc_rows.append((nm, np.nan))
print('Loaded bases:', names)
if len(names) < 3:
    raise RuntimeError('Too few bases for last-block LR.')

OOF=np.column_stack(OOF_list); TEST=np.column_stack(TEST_list)
print('Shapes -> OOF:', OOF.shape, 'TEST:', TEST.shape)

# Train LR on last block only (avoid overfitting to earlier distribution).
sc=StandardScaler(with_mean=True, with_std=True)
Xtr=sc.fit_transform(OOF[last_idx])
ytr=y[last_idx]
C_grid=[0.3, 1.0, 3.0]
best=None
for C in C_grid:
    lr=LogisticRegression(penalty='l2', solver='lbfgs', C=C, max_iter=2000, fit_intercept=True)
    lr.fit(Xtr, ytr)
    z=lr.decision_function(Xtr)
    auc=roc_auc_score(ytr, z)
    print(f'C={C} | AUC_train_last={auc:.5f}')
    if (best is None) or (auc>best[0]): best=(auc,C,lr)
auc_best, C_best, lr_best = best
print('Chosen C:', C_best, '| AUC_train_last=', f'{auc_best:.5f}')

Xte=sc.transform(TEST)
z_gamma = lr_best.decision_function(Xte)
p_gamma = sigmoid(z_gamma).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p_gamma}).to_csv('submission_s51_lastblock_gamma.csv', index=False)
print('Wrote submission_s51_lastblock_gamma.csv | mean', float(p_gamma.mean()))

# Recency on embeddings only (MiniLM, MPNet) using recent35/45; small asymmetric alphas
def load_recent_avg(files):
    arr=[]
    for fp in files:
        if os.path.exists(fp):
            a=np.load(fp).astype(np.float64).ravel(); arr.append(a)
    return np.mean(arr,axis=0) if arr else None
name_to_j = {n:i for i,n in enumerate(names)}
def apply_recency(TEST_base, alphas):
    Z=TEST_base.copy()
    if 'MiniLM' in name_to_j and alphas.get('MiniLM',0)>0:
        pr = load_recent_avg(['test_xgb_emb_meta_time_recent35.npy','test_xgb_emb_meta_time_recent45.npy'])
        if pr is not None:
            j=name_to_j['MiniLM']; Z[:,j]=(1.0-alphas['MiniLM'])*Z[:,j] + alphas['MiniLM']*to_logit(pr)
    if 'MPNet' in name_to_j and alphas.get('MPNet',0)>0:
        pr = load_recent_avg(['test_xgb_emb_mpnet_time_recent35.npy','test_xgb_emb_mpnet_time_recent45.npy'])
        if pr is not None:
            j=name_to_j['MPNet']; Z[:,j]=(1.0-alphas['MPNet'])*Z[:,j] + alphas['MPNet']*to_logit(pr)
    # Keep LR_nosub at 0
    return Z

alphas_low={'MiniLM':0.15,'MPNet':0.20}
alphas_high={'MiniLM':0.25,'MPNet':0.30}
Xte_low = sc.transform(apply_recency(TEST, alphas_low))
Xte_high= sc.transform(apply_recency(TEST, alphas_high))
p_low = sigmoid(lr_best.decision_function(Xte_low)).astype(np.float32)
p_high= sigmoid(lr_best.decision_function(Xte_high)).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p_low}).to_csv('submission_s51_lastblock_r_low.csv', index=False)
pd.DataFrame({id_col: ids, target_col: p_high}).to_csv('submission_s51_lastblock_r_high.csv', index=False)
print('Wrote r_low/r_high means ->', float(p_low.mean()), float(p_high.mean()))

# 3-way logit hedge
zg, zl, zh = to_logit(p_gamma), to_logit(p_low), to_logit(p_high)
p3 = sigmoid((zg+zl+zh)/3.0).astype(np.float32)
pd.DataFrame({id_col: ids, target_col: p3}).to_csv('submission_s51_lastblock_hedge3.csv', index=False)
print('Wrote submission_s51_lastblock_hedge3.csv | mean', float(p3.mean()))

# Bias to 0.30 and 0.32; promote 0.30
def bias_to_mean(probs, target, it=80):
    z = to_logit(probs); lo,hi=-10.0,10.0
    for _ in range(it):
        mid=(lo+hi)/2; m=sigmoid(z+mid).mean()
        if m<target: lo=mid
        else: hi=mid
    return (lo+hi)/2
for tm in [0.30, 0.32]:
    b = bias_to_mean(p3, tm)
    pm = sigmoid(to_logit(p3)+b).astype(np.float32)
    outp = f'submission_s51_lastblock_hedge3_m{int(tm*100):03d}.csv'
    pd.DataFrame({id_col: ids, target_col: pm}).to_csv(outp, index=False)
    print(f'Wrote {outp} | mean={pm.mean():.6f}')
    if abs(tm-0.30)<1e-9:
        pd.DataFrame({id_col: ids, target_col: pm}).to_csv('submission.csv', index=False)
        print('PROMOTED: submission.csv <-', outp)

print('S51-LASTBLOCK-LR done in', f'{time.time()-t0:.1f}s')